In [ ]:
import psutil
def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor
print("="*40, "Memory Information", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}") ; print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}") ; print(f"Percentage: {svmem.percent}%")

======================================== Memory Information ========================================
Total: 12.69GB
Available: 11.83GB
Used: 601.00MB
Percentage: 6.7%


In [ ]:
! nvidia-smi

Mon Jun 28 07:24:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Entity-Relation-Extraction/

/content/drive/MyDrive/Entity-Relation-Extraction


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 30.5MB/s 
     |████████████████████████████████| 901kB 45.0MB/s 
     |████████████████████████████████| 3.3MB 47.5MB/s 


# Model

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel


class MyModel(nn.Module):
    def __init__(self, config):
        super(MyModel, self).__init__()
        self.config = config
        self.bert = AutoModel.from_pretrained(config.pretrained_model_path, return_dict=False)
        self.tag_linear = nn.Linear(self.bert.config.hidden_size, 5)
        self.dropout = nn.Dropout(config.dropout_prob)
        self.loss_func = nn.CrossEntropyLoss()
        # self.theta = config.theta
        self.theta = 0.7

    def forward(self, input, attention_mask, token_type_ids, context_mask=None, turn_mask=None, target_tags=None):
        """
        Args:
            input: （batch,seq_len）
            attention_mask: (batch,seq_len)
            token_type_ids: (batch,seq_len)
            context_mask: (batch,seq_len), used to identify labeled tokens
            target_tags: (batch,seq_len)
            turn_mask: (batch,) turn_mask[i]=0 for turn 1，turn_mask[i]=1 for turn 2
        """
        rep, _ = self.bert(input, attention_mask)
        rep = self.dropout(rep)
        tag_logits = self.tag_linear(rep)  # (batch,seq_len,num_tag)
        if not target_tags is None:
            tag_logits_t1 = tag_logits[turn_mask == 0]  # (n1,seq_len,num_tag)
            target_tags_t1 = target_tags[turn_mask == 0]  # (n1,seq_len)
            context_mask_t1 = context_mask[turn_mask == 0]  # (n1,seq_len)

            tag_logits_t2 = tag_logits[turn_mask == 1]  # (n2,seq_len,num_tag)
            target_tags_t2 = target_tags[turn_mask == 1]  # (n2,seq_len)
            context_mask_t2 = context_mask[turn_mask == 1]  # (n2,seq_len)

            tag_logits_t1 = tag_logits_t1[context_mask_t1 == 1]  # (N1,num_tag)
            target_tags_t1 = target_tags_t1[context_mask_t1 == 1]  # (N1)

            tag_logits_t2 = tag_logits_t2[context_mask_t2 == 1]  # (N2,num_tag)
            target_tags_t2 = target_tags_t2[context_mask_t2 == 1]  # (N2)

            loss_t1 = self.loss_func(tag_logits_t1, target_tags_t1) if len(
                target_tags_t1) != 0 else torch.tensor(0).type_as(input)
            loss_t2 = self.loss_func(tag_logits_t2, target_tags_t2) if len(
                target_tags_t2) != 0 else torch.tensor(0).type_as(input)
            loss = self.theta * loss_t1 + (1-self.theta) * loss_t2
            return loss, (loss_t1.item(), loss_t2.item())
        else:
            # for prediction
            tag_idxs = torch.argmax(tag_logits, dim=-1).squeeze(-1)  # (batch,seq_len)
            return tag_idxs


# Preprocess

In [ ]:
import json

# ad hoc value for frequency of rel(head entity, tail entity)
FREQ = 100 

tag_idxs = {'B': 0, 'M': 1, 'E': 2, 'S': 3, 'O': 4}
zalo_question_templates = json.load(open("zalo_all.json"))


zalo_entities = [
    "CLU", 
    "SCO",
    "PSC",
    "PCA", 
    "PSO",
    "PSI",
    "TSC",
    "TCA", 
    "TSI"
]

zalo_relations = [
    "COMP", # CLU - CLU
    "SCOC", # CLU - SCO
    "SCOP", # PER - CLU 
    "SCOT", # PER - TME
    "CARP", # PER - CLU
    "CART", # PER - TME
    "SUBP", # PER - PER
    "SUBT"  # PER - TME
]

# index of ace2004 and ace2005 frequency matrix
zalo_idx1 = {
    "CLU": 0,
    "SCO": 1, 
    "PSC": 2, 
    "PCA": 3, 
    "PSO": 4, 
    "PSI": 5,
    "TSC": 6,
    "TCA": 7, 
    "TSI": 8
}

zalo_idx2t = {}
for i, rel in enumerate(zalo_relations):
    for j, ent in enumerate(zalo_entities):
        zalo_idx2t[(rel, ent)] = i*len(zalo_entities)+j
zalo_idx2 = zalo_idx2t

# ad hoc value for zalo dist (fake statistics for zalo dataset :D)
zalo_dist = [[0] * len(zalo_entities) * len(zalo_relations) 
                for _ in range(len(zalo_entities))]
print(len(zalo_dist))
print(len(zalo_dist[0]))
zalo_pairs = [
    ("CLU", "CLU"),
    ("CLU", "SCO"),
    ("PSC", "CLU"),
    ("PSC", "TSC"),
    ("PCA", "CLU"),
    ("PCA", "TCA"),
    ("PSI", "PSO"),
    ("PSI", "TSI")
]

for rel, pair in zip(zalo_relations, zalo_pairs):
    head_entity, tail_entity = pair 
    print('head_entity 11', head_entity)
    print('tail_entity', tail_entity)
    print('rel', rel)
    row_idx = zalo_idx1[head_entity]
    print('row_idx', row_idx)
    col_idx = zalo_idx2[(rel, tail_entity)]
    print('col_idx', col_idx)
    zalo_dist[row_idx][col_idx] = FREQ
print('zalo_dist', zalo_dist)
print('zalo_idx2', zalo_idx2)

9
72
head_entity 11 CLU
tail_entity CLU
rel COMP
row_idx 0
col_idx 0
head_entity 11 CLU
tail_entity SCO
rel SCOC
row_idx 0
col_idx 10
head_entity 11 PSC
tail_entity CLU
rel SCOP
row_idx 2
col_idx 18
head_entity 11 PSC
tail_entity TSC
rel SCOT
row_idx 2
col_idx 33
head_entity 11 PCA
tail_entity CLU
rel CARP
row_idx 3
col_idx 36
head_entity 11 PCA
tail_entity TCA
rel CART
row_idx 3
col_idx 52
head_entity 11 PSI
tail_entity PSO
rel SUBP
row_idx 5
col_idx 58
head_entity 11 PSI
tail_entity TSI
rel SUBT
row_idx 5
col_idx 71
zalo_dist [[100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
import torch
import numpy as np
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, DistributedSampler
from transformers import AutoTokenizer

# from preprocess import passage_blocks, get_question
# from constants import *

def passage_blocks(txt, window_size, overlap):
    blocks = []
    regions = []
    for i in range(0, len(txt), window_size-overlap):
        b = txt[i:i+window_size]
        blocks.append(b)
        regions.append((i, i+window_size))
    return blocks, regions


def get_question(question_templates, head_entity, relation_type=None, end_entity_type=None):
    """
    Args:
        head_entity: (entity_type,start_idx,end_idx,entity_string) or entity_type
    """
    if relation_type == None:
        question = question_templates['qa_turn1'][head_entity[0]] if isinstance(
            head_entity, tuple) else question_templates['qa_turn1'][head_entity]
    else:
        question = question_templates['qa_turn2'][str(
            (head_entity[0], relation_type, end_entity_type))]
        question = question.replace('XXX', head_entity[3])
    return question
def collate_fn(batch):
    # for training
    nbatch = {}
    for b in batch:
        for k, v in b.items():
            nbatch[k] = nbatch.get(k, []) + [torch.tensor(v)]

    txt_ids = nbatch['txt_ids']
    tags = nbatch['tags']
    context_mask = nbatch['context_mask']
    token_type_ids = nbatch['token_type_ids']
    turn_mask = nbatch['turn_mask']
    ntxt_ids = pad_sequence(txt_ids, batch_first=True,
                            padding_value=0)
    ntags = pad_sequence(tags, batch_first=True, padding_value=-1)
    ncontext_mask = pad_sequence(
        context_mask, batch_first=True, padding_value=0)
    ntoken_type_ids = pad_sequence(
        token_type_ids, batch_first=True, padding_value=1)
    attention_mask = torch.zeros(ntxt_ids.shape)
    for i in range(len(ntxt_ids)):
        txt_len = len(txt_ids[i])
        attention_mask[i, :txt_len] = 1
    nbatch['txt_ids'] = ntxt_ids
    nbatch['tags'] = ntags
    nbatch['context_mask'] = ncontext_mask
    nbatch['token_type_ids'] = ntoken_type_ids
    nbatch['attention_mask'] = attention_mask
    nbatch['turn_mask'] = torch.tensor(turn_mask, dtype=torch.uint8)
    return nbatch


def collate_fn1(batch):
    # for testing
    nbatch = {}
    for b in batch:
        for k, v in b.items():
            nbatch[k] = nbatch.get(k, []) + [torch.tensor(v)]
    txt_ids = nbatch['txt_ids']
    context_mask = nbatch['context_mask']
    token_type_ids = nbatch['token_type_ids']
    ntxt_ids = pad_sequence(txt_ids, batch_first=True, padding_value=0)
    ncontext_mask = pad_sequence(context_mask, batch_first=True, padding_value=0)
    ntoken_type_ids = pad_sequence(token_type_ids, batch_first=True, padding_value=1)
    attention_mask = torch.zeros(ntxt_ids.shape)
    for i in range(len(ntxt_ids)):
        txt_len = len(txt_ids[i])
        attention_mask[i, :txt_len] = 1
    nbatch['txt_ids'] = ntxt_ids
    nbatch['context_mask'] = ncontext_mask
    nbatch['token_type_ids'] = ntoken_type_ids
    nbatch['attention_mask'] = attention_mask
    return nbatch


def get_inputs(context, q, tokenizer, title="", max_len=200, ans=[], head_entity=None):
    query = tokenizer.tokenize(q)
    tags = [tag_idxs['O']] * len(context)
    for i, an in enumerate(ans):
        start, end = an[1 : -1]
        end = end - 1
        if start != end:
            tags[start] = tag_idxs['B']
            tags[end] = tag_idxs['E']
            for i in range(start + 1, end):
                tags[i] = tag_idxs['M']
        else:
            tags[start] = tag_idxs['S']
    if head_entity:
        h_start, h_end = head_entity[1], head_entity[2]
        context = context[: h_start] + ['[unused0]'] + \
            context[h_start : h_end] + ["[unused1]"] + context[h_end :]
        assert len(context) == len(tags) + 2, f"Expected context length ({len(context)}) == Tags length ({len(tags)}) + 2, context\n {context}"
        tags = tags[: h_start] + [tag_idxs['O']] + \
            tags[h_start : h_end] + [tag_idxs['O']] + tags[h_end :]
    txt_len = len(query) + len(title) + len(context) +  4 if title else len(query) + len(context) + 3
    if txt_len > max_len:
        context = context[: max_len -
                          len(query) - 3] if not title else context[:max_len-len(query)-len(title)-4]
        tags = tags[:max_len -
                    len(query) - 3] if not title else tags[:max_len-len(query)-len(title)-4]

    if title:
        txt = ['<s>'] + query + ['</s>'] + title + ['</s>'] + context + ['</s>']
    else:
        txt = ['<s>'] + query + ['</s>'] + context + ['</s>']
    txt_ids = tokenizer.convert_tokens_to_ids(txt)
    # [CLS] is used to judge whether there is an answer
    if not title:
        tags1 = [tag_idxs['O'] if len(
            ans) > 0 else tag_idxs['S']] + [-1] * (len(query) + 1) + tags + [-1]
        context_mask = [1] + [0] * (len(query) + 1) + [1] * len(context) + [0]
        token_type_ids = [0] * (len(query) + 2) + [1] * (len(context) + 1)
    else:
        tags1 = [tag_idxs['O'] if len(
            ans) > 0 else tag_idxs['S']] + [-1] * (len(query) + len(title) + 2) + tags + [-1]
        context_mask = [1] + [0] * \
            (len(query) + len(title) + 2) + [1] * len(context) + [0]
        token_type_ids = [0] * (len(query) + len(title) + 3) + [1] * (len(context) + 1)
    return txt_ids, tags1, context_mask, token_type_ids


def query2relation(question, question_templates):
    '''
    query -> <entity_type,relation_type,entity_type>
    '''
    # print('question', question)
    turn2_questions = question_templates['qa_turn2']
    # print('turn2_questions', turn2_questions)
    turn2_questions = {v: k for k, v in turn2_questions.items()}
    for k, v in turn2_questions.items():
        k1 = k.split(" XXX ")[1]
        # print('k1', k1)
        if question.find(k1) != -1:
            # print('====', v)
            return eval(v)
    raise Exception("cannot find the relation type corresponding to the query, if the \
                 query template is changed, please re-implement this function according to the new template")


class MyDataset:
    def __init__(self, dataset_tag, path, tokenizer, max_len=256, threshold=5):
        """
        Args:
            dataset_tag: type of dataset
            path:  path to training set file
            tokenizer： tokenizer of pretrained model
            max_len: max length of input
            threshold: only consider relationships where the frequency is greater than or equal to the threshold
        """
        with open(path, encoding='utf-8') as f:
            data = json.load(f)
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.threshold = threshold
        self.dataset_tag = dataset_tag
        self.init_data()

    def init_data(self):
        self.all_t1 = []
        self.all_t2 = []
        if self.dataset_tag.lower() == "zalo":
            idx1s = zalo_idx1
            idx2s = zalo_idx2
            dist = zalo_dist
            question_templates = zalo_question_templates
        elif self.dataset_tag.lower() == "ace2004":
            idx1s = ace2004_idx1
            idx2s = ace2004_idx2
            dist = ace2004_dist
            question_templates = ace2004_question_templates
        elif self.dataset_tag.lower() == 'ace2005':
            idx1s = ace2005_idx1
            idx2s = ace2005_idx2
            dist = ace2005_dist
            question_templates = ace2005_question_templates
        else:
            raise Exception("this dataset is not yet supported")
        for d in tqdm(self.data, desc="dataset"):
            context = d['context']
            title = d['title']
            qa_pairs = d['qa_pairs']
            t1 = qa_pairs[0]
            t2 = qa_pairs[1]
            t1_qas = []
            t2_qas = []
            for i, (q, ans) in enumerate(t1.items()):
                txt_ids, tags, context_mask, token_type_ids = get_inputs(
                    context, q, self.tokenizer, title, self.max_len, ans)
                t1_qas.append({
                    "txt_ids": txt_ids, 
                    "tags": tags,
                    "context_mask": context_mask, 
                    "token_type_ids": token_type_ids, 
                    "turn_mask": 0
                    })

            for t in t2:
                head_entity = t['head_entity']
                for q, ans in t['qas'].items():
                    rel = query2relation(q, question_templates)
                    idx1, idx2 = rel[0], rel[1:]
                    idx1, idx2 = idx1s[idx1], idx2s[idx2]
                    if dist[idx1][idx2] >= self.threshold:
                        txt_ids, tags, context_mask, token_type_ids = get_inputs(
                            context, q, self.tokenizer, title, self.max_len, ans, head_entity)
                        t2_qas.append({
                            "txt_ids": txt_ids, 
                            "tags": tags,
                            "context_mask": context_mask, 
                            "token_type_ids": token_type_ids,
                            "turn_mask" : 1})
            self.all_t1.extend(t1_qas)
            self.all_t2.extend(t2_qas)
        self.all_qas = self.all_t1 + self.all_t2

    def __len__(self):
        return len(self.all_qas)

    def __getitem__(self, i):
        return self.all_qas[i]


class T1Dataset:
    def __init__(self, dataset_tag, test_path, tokenizer, window_size, overlap, max_len=256):
        """
        Args:
            dataset_tag: type of dataset
            test_path: path to test set file
            tokenizer: tokenizer of pretrained model
            window_size: sliding window size
            overlap: overlap between two adjacent windows
            max_len: max length of input
        """
        with open(test_path, encoding="utf=8") as f:
            data = json.load(f)
        self.dataset_tag = dataset_tag
        if dataset_tag.lower() == 'zalo':
            dataset_entities = zalo_entities
            question_templates = zalo_question_templates
        elif dataset_tag.lower() == 'ace2004':
            dataset_entities = ace2004_entities
            question_templates = ace2004_question_templates
        elif dataset_tag.lower() == 'ace2005':
            dataset_entities = ace2005_entities
            question_templates = ace2005_question_templates
        else:
            raise Exception("this data set is not yet supported")
        self.t1_qas = []
        self.passages = []
        self.entities = []
        self.relations = []
        self.titles = []
        self.window_size = window_size
        self.overlap = overlap
        self.t1_querys = []
        self.t1_ids = []
        self.t1_gold = []
        self.ids = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        # passage_windows[i][j] represents the j-th window of the i-th passage
        self.passage_windows = []
        self.query_offset1 = []
        self.window_offset_base = window_size-overlap

        for ent_type in dataset_entities:
            query = get_question(question_templates, ent_type)
            self.t1_querys.append(query)

        for p_id, d in enumerate(tqdm(data, desc="t1_dataset")):
            passage = d["passage"]
            entities = d['entities']
            relations = d['relations']
            title = d['title']
            self.ids.append(d["id"])
            self.passages.append(passage)
            self.entities.append(entities)
            self.relations.append(relations)
            self.titles.append(title)
            blocks, _ = passage_blocks(passage, window_size, overlap)
            self.passage_windows.append(blocks)
            for ent in entities:
                self.t1_gold.append((p_id, tuple(ent[:-1])))
            for b_id, block in enumerate(blocks):
                for q_id, q in enumerate(self.t1_querys):
                    txt_ids, _, context_mask, token_type_ids = get_inputs(
                        block, q, tokenizer, title, max_len)
                    self.t1_qas.append({"txt_ids": txt_ids, "context_mask": context_mask,
                                        "token_type_ids": token_type_ids})
                    self.t1_ids.append((p_id, b_id, dataset_entities[q_id]))
                    if not title:
                        ofs = len(tokenizer.tokenize(q)) + 2
                    else:
                        ofs = len(title) + len(tokenizer.tokenize(q)) + 3
                    self.query_offset1.append(ofs)
                    
    def __len__(self):
        return len(self.t1_qas)

    def __getitem__(self, i):
        return self.t1_qas[i]


class T2Dataset:
    def __init__(self, t1_dataset, t1_predict, threshold=5):
        '''
        Args:
            t1_dataset: an instance of T1Dataset
            t1_predict: predictions of the first turn QA
            threshold: only consider relationships where the frequency is greater than or equal to the threshold
        '''
        if t1_dataset.dataset_tag.lower() == "zalo":
            idx1s = zalo_idx1
            idx2s = zalo_idx2
            dist = zalo_dist
            dataset_entities = zalo_entities
            dataset_relations = zalo_relations
            question_templates = zalo_question_templates
        elif t1_dataset.dataset_tag.lower() == "ace2004":
            idx1s = ace2004_idx1
            idx2s = ace2004_idx2
            dist = ace2004_dist
            dataset_entities = ace2004_entities
            dataset_relations = ace2004_relations
            question_templates = ace2004_question_templates
        elif t1_dataset.dataset_tag.lower() == 'ace2005':
            idx1s = ace2005_idx1
            idx2s = ace2005_idx2
            dist = ace2005_dist
            dataset_entities = ace2005_entities
            dataset_relations = ace2005_relations
            question_templates = ace2005_question_templates
        else:
            raise Exception("this data set is not yet supported")
        tokenizer = t1_dataset.tokenizer
        max_len = t1_dataset.max_len
        t1_ids = t1_dataset.t1_ids
        passages = t1_dataset.passages
        titles = t1_dataset.titles
        passage_windows = t1_dataset.passage_windows
        self.t2_qas = []
        self.t2_ids = []
        self.t2_gold = []
        self.query_offset2 = []
        relations = t1_dataset.relations
        entities = t1_dataset.entities
        query_offset1 = t1_dataset.query_offset1
        window_offset_base = t1_dataset.window_offset_base

        for passage_id, (ents, rels) in enumerate(zip(entities, relations)):
            for re in rels:
                head, rel, end = ents[re[1]], re[0], ents[re[2]]
                self.t2_gold.append(
                    (passage_id, (tuple(head[:-1]), rel, tuple(end[:-1]))))

        for i, (_id, pre) in enumerate(zip(tqdm(t1_ids, desc="t2 dataset"), t1_predict)):
            passage_id, window_id, head_entity_type = _id
            window_offset = window_offset_base * window_id
            context = passage_windows[passage_id][window_id]
            title = titles[passage_id]
            head_entities = []
            for start, end in pre:
                start1, end1 = start - \
                    query_offset1[i] + window_offset, end - \
                    query_offset1[i] + window_offset
                ent_str = tokenizer.convert_tokens_to_string(
                    passages[passage_id][start1:end1])
                head_entity = (head_entity_type, start1, end1, ent_str)
                head_entities.append(head_entity)

            for head_entity in head_entities:
                for rel in dataset_relations:
                    for end_ent_type in dataset_entities:
                        idx1, idx2 = idx1s[head_entity[0]], idx2s[(rel, end_ent_type)]
                        if dist[idx1][idx2] >= threshold:
                            query = get_question(
                                question_templates, head_entity, rel, end_ent_type)
                            window_head_entity = (
                                head_entity[0], head_entity[1]-window_offset, head_entity[2]-window_offset, head_entity[3])
                            txt_ids, _, context_mask, token_type_ids = get_inputs(
                                context, query, tokenizer, title, max_len, [], window_head_entity)
                            self.t2_qas.append({"txt_ids": txt_ids, "context_mask": context_mask,
                                                "token_type_ids": token_type_ids})
                            self.t2_ids.append(
                                (passage_id, window_id, head_entity[:-1], rel, end_ent_type))
                            if not title:
                                ofs = len(tokenizer.tokenize(query)) + 2
                            else:
                                ofs = len(title) + len(tokenizer.tokenize(query)) + 3 
                            self.query_offset2.append(ofs)

    def __len__(self):
        return len(self.t2_qas)

    def __getitem__(self, i):
        return self.t2_qas[i]


def load_data(dataset_tag, file_path, batch_size, max_len, pretrained_model_path, dist=False, shuffle=False, threshold=5):
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path)
    dataset = MyDataset(dataset_tag, file_path, tokenizer, max_len, threshold)
    sampler = DistributedSampler(dataset) if dist else None
    dataloader = DataLoader(dataset, batch_size, sampler=sampler, shuffle=shuffle if not sampler else False, collate_fn=collate_fn, drop_last=True)
    return dataloader


def reload_data(old_dataloader, batch_size, max_len, threshold, local_rank, shuffle=True):
    dataset = old_dataloader.dataset
    old_max_len, old_threshold = dataset.max_len, dataset.threshold
    if not (old_max_len == max_len and old_threshold == threshold):
        dataset.max_len = max_len
        dataset.threshold = threshold
        dataset.init_data()
    sampler = DistributedSampler(
        dataset, rank=local_rank) if local_rank != -1 else None
    dataloader = DataLoader(
        dataset, batch_size, sampler=sampler, shuffle=shuffle, collate_fn=collate_fn, drop_last=True)
    return dataloader


def load_t1_data(dataset_tag, test_path, pretrained_model_path, window_size, overlap, batch_size=10, max_len=256):
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path)
    t1_dataset = T1Dataset(dataset_tag, test_path, tokenizer, window_size, overlap, max_len)
    dataloader = DataLoader(t1_dataset, batch_size, collate_fn=collate_fn1, drop_last=True)
    return dataloader


def load_t2_data(t1_dataset, t1_predict, batch_size=10, threshold=5):
    t2_dataset = T2Dataset(t1_dataset, t1_predict, threshold)
    dataloader = DataLoader(t2_dataset, batch_size, collate_fn=collate_fn1, drop_last=True)
    return dataloader


In [ ]:
import os
import time
import random
import argparse

import pickle
from transformers.optimization import get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import autocast, GradScaler
from tqdm import trange, tqdm
import numpy as np
import torch
from torch.nn.utils import clip_grad_norm_

# from model import MyModel
# from evaluation import test_evaluation
# from dataloader import load_data, load_t1_data, reload_data

In [ ]:
import torch
from tqdm import tqdm

# from dataloader import tag_idxs, load_t2_data


def get_score(gold_set, predict_set):
    TP = len(set.intersection(gold_set, predict_set))
    print("#TP:", TP, "#Gold:", len(gold_set), "#Predict:", len(predict_set))
    precision = TP/(len(predict_set)+1e-6)
    recall = TP/(len(gold_set)+1e-6)
    f1 = 2*precision*recall/(precision+recall+1e-6)
    return precision, recall, f1


def test_evaluation(model, t1_dataloader, threshold, amp=False):
    if hasattr(model, 'module'):
        model = model.module
    model.eval()
    t1_predict = []
    t2_predict = []
    device = torch.device(
        "cuda") if torch.cuda.is_available() else torch.device("cpu")

    # turn 1
    with (torch.no_grad() if not amp else torch.cuda.amp.autocast()):
        for i, batch in enumerate(tqdm(t1_dataloader, desc="t1 predict")):
            txt_ids, attention_mask, token_type_ids, context_mask = batch['txt_ids'], batch[
                'attention_mask'], batch['token_type_ids'], batch['context_mask']
            tag_idxs = model(
                    txt_ids.to(device), 
                    attention_mask.to(device), 
                    token_type_ids.to(device)
                    )
            predict_spans = tag_decode(tag_idxs, context_mask)
            t1_predict.extend(predict_spans)

    # turn 2
    t2_dataloader = load_t2_data(
        t1_dataloader.dataset, t1_predict, 10, threshold)
    with (torch.no_grad() if not amp else torch.cuda.amp.autocast()):
        for i, batch in enumerate(tqdm(t2_dataloader, desc="t2 predict")):
            txt_ids, attention_mask, token_type_ids, context_mask = batch['txt_ids'], batch[
                'attention_mask'], batch['token_type_ids'], batch['context_mask']
            tag_idxs = model(txt_ids.to(device), attention_mask.to(device), token_type_ids.to(device))
            predict_spans = tag_decode(tag_idxs, context_mask)
            t2_predict.extend(predict_spans)

    # get basic information
    t1_ids = t1_dataloader.dataset.t1_ids
    t2_ids = t2_dataloader.dataset.t2_ids
    window_offset_base = t1_dataloader.dataset.window_offset_base
    query_offset1 = t1_dataloader.dataset.query_offset1
    query_offset2 = t2_dataloader.dataset.query_offset2
    t1_gold = t1_dataloader.dataset.t1_gold
    t2_gold = t2_dataloader.dataset.t2_gold
    p1, r1, f1 = eval_t1(t1_predict, t1_gold, t1_ids,
                        query_offset1, window_offset_base)
    p2, r2, f2 = eval_t2(t2_predict, t2_gold, t2_ids,
                         query_offset2, window_offset_base)
    return (p1, r1, f1), (p2, r2, f2)


def eval_t1(predict, gold, ids, query_offset, window_offset_base):
    """
    Args:
        predict: [(s1,e1),(s2,e2),(s3,e3),...]
        gold: (passage_id,(entity_type,start_idx,end_idx,entity_str))
        ids: (passage_id, window_id,entity_type)
        query_offset: offset of [CLS]+title+[SEP]+query+[SEP]
        window_offset_base: value of window_size-overlap的值
    """
    predict1 = []
    for i, (_id, pre) in enumerate(zip(ids, predict)):
        passage_id, window_id, entity_type = _id
        window_offset = window_offset_base * window_id
        for start, end in pre:
            start1, end1 = start - query_offset[i] + window_offset, \
                            end - query_offset[i] + window_offset
            new = (passage_id, (entity_type, start1, end1))
            predict1.append(new)
    return get_score(set(gold), set(predict1))


def eval_t2(predict, gold, ids, query_offset, window_offset_base):
    """
    Args:
        predict: [(s1,e1),(s2,e2),(s3,e3),...]
        gold:  (passage_id,(head_entity,relation_type,end_entity))
        ids: (passage_id,window_id,head_entity,relation_type,end_entity_type)
        query_offset: offset of [CLS]+title+[SEP]+query+[SEP]
        window_offset_base: value of window_size-overlap
    """
    predict1 = []
    for i, (_id, pre) in enumerate(zip(ids, predict)):
        passage_id, window_id, head_entity, relation_type, end_entity_type = _id
        window_offset = window_offset_base * window_id
        head_start = head_entity[1]
        for start, end in pre:
            #since we added two special tokens around the head entity for identification, there is a correction of 1.
            if head_start+query_offset[i]-window_offset+1 < start:
                start1, end1 = start - \
                    query_offset[i]+window_offset-2, end - \
                    query_offset[i]+window_offset-2
            else:
                start1, end1 = start - \
                    query_offset[i]+window_offset, end - \
                    query_offset[i]+window_offset
            new = (passage_id, (head_entity, relation_type,
                                (end_entity_type, start1, end1)))
            predict1.append(new)
    return get_score(set(gold), set(predict1))


def tag_decode(tags, context_mask=None):
    spans = [[]] * tags.shape[0]
    tags = tags.tolist()
    if not context_mask is None:
        context_mask = context_mask.tolist()
    has_answer = []
    start_idxs = []
    end_idxs = []
    for i, t in enumerate(tags):
        if t[0] != tag_idxs['S']:
            has_answer.append(i)
            if context_mask is None:
                mask = [1 if i != -1 else 0 for i in t]
            else:
                mask = context_mask[i]
            s = mask.index(1, 1)
            e = mask.index(0, s)
            start_idxs.append(s)
            end_idxs.append(e)
    for i, s, e in zip(has_answer, start_idxs, end_idxs):
        span = []
        j = s
        while j < e:
            if tags[i][j] == tag_idxs['S']:
                span.append([j, j + 1])
                j += 1
            elif tags[i][j] == tag_idxs['B'] and j < e-1:
                for k in range(j + 1, e):
                    if tags[i][k] in [tag_idxs['B'], tag_idxs['S']]:
                        j = k
                        break
                    elif tags[i][k] == tag_idxs["E"]:
                        span.append([j, k + 1])
                        j = k + 1
                        break
                    elif k == e - 1:
                        j = k + 1
            else:
                j += 1
        spans[i] = span
    return spans

# Train

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

def train(args, train_dataloader):
    model = MyModel(args)
    model.train()
    if args.amp:
        scaler = GradScaler()
    print('0')
    device = args.local_rank if args.local_rank != -1 \
        else (torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu'))
    if args.local_rank != -1:
        torch.cuda.set_device(args.local_rank)
    model.to(device)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[
                                                          args.local_rank], output_device=args.local_rank, find_unused_parameters=True)
    no_decay = ['bias', 'LayerNorm.weight']
    print('1')
    optimizer_grouped_parameters = [
        {"params": [p for n, p in model.named_parameters() if not any(
            nd in n for nd in no_decay)], "weight_decay":args.weight_decay},
        {"params": [p for n, p in model.named_parameters() if any(
            nd in n for nd in no_decay)], "weight_decay":0.0}
    ]
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=args.lr)
    if args.warmup_ratio > 0:
        num_training_steps = len(train_dataloader)*args.max_epochs
        warmup_steps = args.warmup_ratio*num_training_steps
        scheduler = get_linear_schedule_with_warmup(
            optimizer, warmup_steps, num_training_steps)
    if args.local_rank < 1:
        mid = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime(time.time()))
    for epoch in range(args.max_epochs):
        if args.local_rank != -1:
            train_dataloader.sampler.set_epoch(epoch)
        tqdm_dict = {"loss": "0.00", "loss_t1": "0.00", "loss_t2": "0.00"}
        tqdm_train_dataloader = tqdm(
            train_dataloader, desc="epoch:%d" % epoch, ncols=100, postfix=tqdm_dict)
        for i, batch in enumerate(tqdm_train_dataloader):
            torch.cuda.empty_cache()
            optimizer.zero_grad()
            txt_ids, attention_mask, token_type_ids, context_mask, turn_mask, tags = batch['txt_ids'], batch['attention_mask'], batch['token_type_ids'],\
                batch['context_mask'], batch['turn_mask'], batch['tags']
            txt_ids, attention_mask, token_type_ids, context_mask, turn_mask, tags = txt_ids.to(device), attention_mask.to(device), token_type_ids.to(device),\
                context_mask.to(device), turn_mask.to(device), tags.to(device)
            if args.amp:
                with autocast():
                    loss, (loss_t1, loss_t2) = model(
                        txt_ids, attention_mask, token_type_ids, context_mask, turn_mask, tags)
                scaler.scale(loss).backward()
                if args.max_grad_norm > 0:
                    scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(
                    model.parameters(), args.max_grad_norm)
                scaler.step(optimizer)
                scaler.update()
            else:
                loss, (loss_t1, loss_t2) = model(txt_ids, attention_mask,
                                                 token_type_ids, context_mask, turn_mask, tags)
                loss.backward()
                if args.max_grad_norm > 0:
                    clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
            lr = optimizer.param_groups[0]['lr']
            named_parameters = [
                (n, p) for n, p in model.named_parameters() if not p.grad is None]
            grad_norm = torch.norm(torch.stack(
                [torch.norm(p.grad) for n, p in named_parameters])).item()
            if args.warmup_ratio > 0:
                scheduler.step()

            tqdm_dict["loss"] = "{:.2f}".format(loss.item())
            tqdm_dict["loss_t1"] = "{:.2f}".format(loss_t1)
            tqdm_dict["loss_t2"] = "{:.2f}".format(loss_t2)
            tqdm_train_dataloader.set_postfix(tqdm_dict)

        if args.local_rank in [-1, 0] and not args.not_save:
            if hasattr(model, 'module'):
                model_state_dict = model.module.state_dict()
            else:
                model_state_dict = model.state_dict()
            checkpoint = {"model_state_dict": model_state_dict}
            save_dir = './checkpoints/%s/%s/' % (args.dataset_tag, mid)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
                pickle.dump(args, open(save_dir+'args', 'wb'))
            save_path = save_dir+"checkpoint_%d.ckpt" % epoch
            torch.save(checkpoint, save_path)
            print("model saved at:", save_path)

        if args.test_eval and args.local_rank in [-1, 0]:
            test_dataloader = load_t1_data(args.dataset_tag, args.test_path, args.pretrained_model_path,
                                           args.window_size, args.overlap, args.test_batch, args.max_len)  # test_dataloader是第一轮问答的dataloder
            (p1, r1, f1), (p2, r2, f2) = test_evaluation(
                model, test_dataloader, args.threshold, args.amp)
            print(
                "Turn 1: precision:{:.4f} recall:{:.4f} f1:{:.4f}".format(p1, r1, f1))
            print(
                "Turn 2: precision:{:.4f} recall:{:.4f} f1:{:.4f}".format(p2, r2, f2))
            model.train()
        if args.local_rank != -1:
            torch.distributed.barrier()

In [ ]:
class Arg:
    def __init__(self):
        # argss = {}
        self.dropout_prob = 0.1
        self.local_rank = -1
        self.weight_decay = 0.01
        self.warmup_ratio = 0.1
        self.max_epochs = 5
        self.lr = 2e-5
        self.amp = True
        self.max_grad_norm = 1
        self.test_eval = False
        self.threshold = 1
        self.dataset_tag = 'zalo'
        self.max_len = 256
        self.window_size = 220
        self.overlap = 45
        self.max_grad_norm = 1
        self.test_batch = 10
        self.train_batch = 10
        self.theta = 0.25
        self.seed = 0
        self.not_save = False
        self.pretrained_model_path = "vinai/phobert-base"
        self.test_path = "dataset/dev.json"
        self.train_path = "dataset/train.json"
        self.checkpoint_path = "./checkpoints/zalo/2021_06_28_01_37_33/checkpoint_4.ckpt"
# if __name__ == "__main__":
argss = Arg()
# train_dataloader = load_data(argss.dataset_tag, argss.train_path, argss.train_batch, argss.max_len, argss.pretrained_model_path,
#                               False, shuffle=True, threshold=argss.threshold)
# print(torch.cuda.is_available())
# train(argss, train_dataloader)

# Test

In [ ]:
import os
import argparse
import pickle
import torch

# from evaluation import test_evaluation
# from model import MyModel
# from dataloader import load_t1_data

# model_dir,file = os.path.split(args.checkpoint_path)
# config = pickle.load(open(os.path.join(model_dir,'args'),'rb'))
print(argss.checkpoint_path)
checkpoint = torch.load(argss.checkpoint_path,map_location=torch.device("cpu"))
model_state_dict = checkpoint['model_state_dict']
#replace default vaule if given
# config.pretrained_model_path = args.pretrained_model_path if args.pretrained_model_path else config.pretrained_model_path
# config.threshold = args.threshold if args.threshold==-1 else config.threshold
mymodel = MyModel(argss)
mymodel.load_state_dict(model_state_dict,strict=False)
device = torch.device("cuda") if  torch.cuda.is_available() else torch.device("cpu")
mymodel.to(device)

test_dataloader = load_t1_data(argss.dataset_tag,argss.test_path,argss.pretrained_model_path,220,45,argss.test_batch,argss.max_len)
(p1,r1,f1),(p2,r2,f2) = test_evaluation(mymodel,test_dataloader,argss.threshold,False)
print("Turn 1: precision:{:.4f} recall:{:.4f} f1:{:.4f}".format(p1,r1,f1))
print("Turn 2: precision:{:.4f} recall:{:.4f} f1:{:.4f}".format(p2,r2,f2))

./checkpoints/zalo/2021_06_28_01_37_33/checkpoint_4.ckpt


Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
t2 predict: 100%|██████████| 343/343 [00:55<00:00,  6.19it/s]

#TP: 1642 #Gold: 2171 #Predict: 2536
#TP: 763 #Gold: 1367 #Predict: 1578
Turn 1: precision:0.6475 recall:0.7563 f1:0.6977
Turn 2: precision:0.4835 recall:0.5582 f1:0.5182


# Visualization

In [ ]:
import torch
from tqdm import tqdm

# from dataloader import tag_idxs, load_t2_data


def get_score(gold_set, predict_set):
    TP = len(set.intersection(gold_set, predict_set))
    print("#TP:", TP, "#Gold:", len(gold_set), "#Predict:", len(predict_set))
    precision = TP/(len(predict_set)+1e-6)
    recall = TP/(len(gold_set)+1e-6)
    f1 = 2*precision*recall/(precision+recall+1e-6)
    return precision, recall, f1


# def test_evaluation(model, t1_dataloader, threshold, amp=False):
#     t1_predict, t2_predict = inference(model, t1_dataloader, threshold, amp=False)
#     t1_gold = t1_dataloader.dataset.t1_gold
#     t2_gold = t2_dataloader.dataset.t2_gold
#     p1, r1, f1 = eval_t1(t1_predict, t1_gold)
#     p2, r2, f2 = eval_t2(t2_predict, t2_gold)
#     return (p1, r1, f1), (p2, r2, f2)

def inference(model, t1_dataloader, threshold, amp=False):
    if hasattr(model, 'module'):
        model = model.module
    model.eval()
    t1_predict = []
    t2_predict = []
    device = torch.device(
        "cuda") if torch.cuda.is_available() else torch.device("cpu")

    # turn 1
    with (torch.no_grad() if not amp else torch.cuda.amp.autocast()):
        for i, batch in enumerate(tqdm(t1_dataloader, desc="t1 predict")):
            txt_ids, attention_mask, token_type_ids, context_mask = batch['txt_ids'], batch[
                'attention_mask'], batch['token_type_ids'], batch['context_mask']
            tag_idxs = model(
                    txt_ids.to(device), 
                    attention_mask.to(device), 
                    token_type_ids.to(device)
                    )
            predict_spans = tag_decode(tag_idxs, context_mask)
            t1_predict.extend(predict_spans)

    # turn 2
    t2_dataloader = load_t2_data(
        t1_dataloader.dataset, t1_predict, 10, threshold)
    with (torch.no_grad() if not amp else torch.cuda.amp.autocast()):
        for i, batch in enumerate(tqdm(t2_dataloader, desc="t2 predict")):
            txt_ids, attention_mask, token_type_ids, context_mask = batch['txt_ids'], batch[
                'attention_mask'], batch['token_type_ids'], batch['context_mask']
            tag_idxs = model(txt_ids.to(device), attention_mask.to(device), token_type_ids.to(device))
            predict_spans = tag_decode(tag_idxs, context_mask)
            t2_predict.extend(predict_spans)

    # get basic information
    t1_ids = t1_dataloader.dataset.t1_ids
    t2_ids = t2_dataloader.dataset.t2_ids
    window_offset_base = t1_dataloader.dataset.window_offset_base
    query_offset1 = t1_dataloader.dataset.query_offset1
    query_offset2 = t2_dataloader.dataset.query_offset2
    t1_predict = predict_t1(t1_predict, t1_ids, query_offset1, window_offset_base)
    t2_predict = predict_t2(t2_predict, t2_ids, query_offset2, window_offset_base)
    return t1_predict, t2_predict

def predict_t1(predict, ids, query_offset, window_offset_base):
    predict1 = []
    for i, (_id, pre) in enumerate(zip(ids, predict)):
        passage_id, window_id, entity_type = _id
        window_offset = window_offset_base * window_id
        for start, end in pre:
            start1, end1 = start - query_offset[i] + window_offset, \
                            end - query_offset[i] + window_offset
            new = (passage_id, (entity_type, start1, end1))
            predict1.append(new)
    return predict1    

def eval_t1(predict, gold, ids, query_offset, window_offset_base):
    """
    Args:
        predict: [(s1,e1),(s2,e2),(s3,e3),...]
        gold: (passage_id,(entity_type,start_idx,end_idx,entity_str))
        ids: (passage_id, window_id,entity_type)
        query_offset: offset of [CLS]+title+[SEP]+query+[SEP]
        window_offset_base: value of window_size-overlap的值
    """
    return get_score(set(gold), set(predict1))

def predict_t2(predict, ids, query_offset, window_offset_base):
    predict1 = []
    for i, (_id, pre) in enumerate(zip(ids, predict)):
        passage_id, window_id, head_entity, relation_type, end_entity_type = _id
        window_offset = window_offset_base * window_id
        head_start = head_entity[1]
        for start, end in pre:
            #since we added two special tokens around the head entity for identification, there is a correction of 1.
            if head_start+query_offset[i]-window_offset+1 < start:
                start1, end1 = start - \
                    query_offset[i]+window_offset-2, end - \
                    query_offset[i]+window_offset-2
            else:
                start1, end1 = start - \
                    query_offset[i]+window_offset, end - \
                    query_offset[i]+window_offset
            new = (passage_id, (head_entity, relation_type,
                                (end_entity_type, start1, end1)))
            predict1.append(new)
    return predict1

def eval_t2(predict, gold, ids, query_offset, window_offset_base):
    """
    Args:
        predict: [(s1,e1),(s2,e2),(s3,e3),...]
        gold:  (passage_id,(head_entity,relation_type,end_entity))
        ids: (passage_id,window_id,head_entity,relation_type,end_entity_type)
        query_offset: offset of [CLS]+title+[SEP]+query+[SEP]
        window_offset_base: value of window_size-overlap
    """
    return get_score(set(gold), set(predict1))

def tag_decode(tags, context_mask=None):
    spans = [[]] * tags.shape[0]
    tags = tags.tolist()
    if not context_mask is None:
        context_mask = context_mask.tolist()
    has_answer = []
    start_idxs = []
    end_idxs = []
    for i, t in enumerate(tags):
        if t[0] != tag_idxs['S']:
            has_answer.append(i)
            if context_mask is None:
                mask = [1 if i != -1 else 0 for i in t]
            else:
                mask = context_mask[i]
            s = mask.index(1, 1)
            e = mask.index(0, s)
            start_idxs.append(s)
            end_idxs.append(e)
    for i, s, e in zip(has_answer, start_idxs, end_idxs):
        span = []
        j = s
        while j < e:
            if tags[i][j] == tag_idxs['S']:
                span.append([j, j + 1])
                j += 1
            elif tags[i][j] == tag_idxs['B'] and j < e-1:
                for k in range(j + 1, e):
                    if tags[i][k] in [tag_idxs['B'], tag_idxs['S']]:
                        j = k
                        break
                    elif tags[i][k] == tag_idxs["E"]:
                        span.append([j, k + 1])
                        j = k + 1
                        break
                    elif k == e - 1:
                        j = k + 1
            else:
                j += 1
        spans[i] = span
    return spans

In [ ]:
# config.threshold = 1
tokenizer = AutoTokenizer.from_pretrained(argss.pretrained_model_path)
print(argss.checkpoint_path)
mymodel = MyModel(argss)
mymodel.load_state_dict(model_state_dict,strict=False)
device = torch.device("cuda") if  torch.cuda.is_available() else torch.device("cpu")
mymodel.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


./checkpoints/zalo/2021_06_28_01_37_33/checkpoint_4.ckpt


Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MyModel(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=

In [ ]:
test_dataloader = load_t1_data(argss.dataset_tag, 
                               "dataset/test.json",
                               argss.pretrained_model_path,
                               220,
                               45,
                               10,
                               256)
t1_predict, t2_predict = inference(mymodel,test_dataloader,argss.threshold, False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
t2 predict: 100%|██████████| 536/536 [01:25<00:00,  6.24it/s]


In [ ]:
passages = test_dataloader.dataset.passages
ids = test_dataloader.dataset.ids
entities, relations = {}, {}
print(t1_predict)
print(t2_predict)
for p_id, (ner_type, start_pos, end_pos) in t1_predict:
    if not p_id in entities:
        entities[p_id] = []
    text = passages[p_id][start_pos : end_pos] 
    entities[p_id].append((ner_type, tokenizer.convert_tokens_to_string(text)))

for p_id, ((head_entity_type, head_start, head_stop), relation_type, (end_entity_type, end_start, end_stop)) in t2_predict:
    if not p_id in relations:
        relations[p_id] = []
    end_entity = passages[p_id][head_start : head_stop]
    head_entity = passages[p_id][end_start : end_stop]
    end_entity = tokenizer.convert_tokens_to_string(end_entity)
    head_entity = tokenizer.convert_tokens_to_string(head_entity)
    relations[p_id].append((relation_type, head_entity, end_entity))

for p_id in range(len(passages)):
    with open("./predictions/{}.txt".format(ids[p_id]), "w") as fin:
        if p_id in entities:
            for entity in entities[p_id]:
                fin.write("{}\t{}\n".format(entity[0], entity[1]))
        if p_id in relations:
            for relation in relations[p_id]:
                fin.write("{}\t{}\t{}\n".format(relation[0], relation[2], relation[1]))

[(0, ('CLU', 74, 75)), (0, ('CLU', 196, 197)), (0, ('CLU', 196, 197)), (0, ('CLU', 275, 276)), (0, ('PSC', 268, 270)), (0, ('PSI', 330, 333)), (0, ('TSC', 234, 235)), (0, ('CLU', 473, 474)), (0, ('PSC', 478, 479)), (0, ('PSC', 509, 510)), (0, ('TSC', 507, 508)), (0, ('CLU', 695, 696)), (0, ('PSC', 645, 646)), (0, ('PSC', 713, 714)), (0, ('TSC', 617, 618)), (0, ('SCO', 705, 706)), (0, ('PSC', 713, 714)), (1, ('CLU', 0, 1)), (1, ('CLU', 30, 33)), (1, ('CLU', 66, 67)), (1, ('CLU', 212, 213)), (1, ('PSC', 73, 74)), (1, ('PSC', 153, 154)), (1, ('TSC', 86, 87)), (1, ('TSC', 163, 164)), (1, ('SCO', 231, 232)), (1, ('SCO', 233, 234)), (2, ('CLU', 22, 23)), (2, ('CLU', 189, 190)), (2, ('CLU', 189, 190)), (2, ('CLU', 282, 283)), (2, ('CLU', 432, 433)), (2, ('PSC', 418, 422)), (2, ('PSI', 515, 516)), (2, ('PSI', 517, 519)), (2, ('TSC', 396, 398)), (2, ('CLU', 627, 628)), (2, ('SCO', 623, 624)), (2, ('SCO', 625, 626)), (2, ('PSC', 595, 601)), (2, ('TSC', 630, 631)), (3, ('CLU', 5, 7)), (3, ('CLU',

In [ ]:
# print(passages)
for p_id in range(len(passages)):
    with open("./predictions/{}.txt".format(ids[p_id]), "r", encoding='utf-8') as f:
        raw_txt = f.read()
        txt = [t for t in raw_txt.split('\n') if t.strip()]
        # print(ids[p_id])
        if 'SCOP' in raw_txt:
          print(ids[p_id])
          print(raw_txt)
# with open("./predictions/22270589.txt", "r", encoding='utf-8') as f:
#     raw_txt = f.read()
#     txt = [t for t in raw_txt.split('\n') if t.strip()]
#     print(txt)
#     if 'SCOT' in raw_txt:
#       print(raw_txt)

In [ ]:
for p_id in range(len(passages)):
    with open("./predictions/{}.txt".format(ids[p_id]), "w") as fin:
      print(fin)
        # if p_id in entities:
        #     for entity in entities[p_id]:
        #         fin.write("{}\t{}\n".format(entity[0], entity[1]))

In [ ]:
import json
from collections import defaultdict
from os import listdir
from os.path import isfile, join

test_result = []
prediction_dir = "predictions"
files = [f for f in listdir(prediction_dir) if isfile(join(prediction_dir, f))]
files

['21464024.txt',
 '21468899.txt',
 '21464252.txt',
 '21488329.txt',
 '21468455.txt',
 '21464178.txt',
 '21517172.txt',
 '21502158.txt',
 '21512620.txt',
 '21502154.txt',
 '21495046.txt',
 '21518905.txt',
 '21518573.txt',
 '21519193.txt',
 '21520713.txt',
 '21518158.txt',
 '21521432.txt',
 '21521494.txt',
 '21522916.txt',
 '21523378.txt',
 '21523609.txt',
 '21526074.txt',
 '21528640.txt',
 '21533054.txt',
 '21532871.txt',
 '21533042.txt',
 '21541461.txt',
 '21534660.txt',
 '21541830.txt',
 '21542202.txt',
 '21542083.txt',
 '21545075.txt',
 '21543456.txt',
 '21542304.txt',
 '21543348.txt',
 '21550261.txt',
 '21550116.txt',
 '21550191.txt',
 '21550577.txt',
 '21550903.txt',
 '21552496.txt',
 '21557605.txt',
 '21552950.txt',
 '21559003.txt',
 '21561253.txt',
 '21560540.txt',
 '21572994.txt',
 '21572831.txt',
 '21574797.txt',
 '21575047.txt',
 '21573925.txt',
 '21575304.txt',
 '21578873.txt',
 '21579778.txt',
 '21581258.txt',
 '21597633.txt',
 '21598068.txt',
 '21597761.txt',
 '21598130.txt

In [ ]:
file = files[10]
file
entities = defaultdict(set)
relations = defaultdict(set)
match_summary = {}
test_id = file.split('/')[-1].split('.')[0]
print(test_id)
with open(join(prediction_dir, files[10]), 'r') as f:
    lines = f.readlines()
    print(lines)
    
file

21495046
['CLU\tLeicester_City\n', 'CLU\tDerby_County\n', 'CLU\tDerby_County\n', 'PSC\tAndy_King\n', 'PSC\tRazzagui_Camara\n', 'PSC\tOnyinye_Ndidi\n', 'TSC\thiệp 2 diễn ra được 2 phút\n', 'TSC\t61\n', 'CLU\tLeicester\n', 'CLU\tLeicester_City\n', 'SCO\t3\n', 'SCO\t1\n', 'PSC\tOnyinye_Ndidi\n', 'PSC\tDemarai_Gray\n', 'TSC\t95\n', 'TSC\tTrước khi trận đấu khép lại\n', 'COMP\tLeicester_City\tDerby_County\n', 'COMP\tDerby_County\tLeicester\n', 'SCOP\tAndy_King\tDerby_County\n', 'SCOT\tAndy_King\thiệp 2 diễn ra được 2 phút\n', 'SCOP\tRazzagui_Camara\tDerby_County\n', 'SCOT\tRazzagui_Camara\t61\n', 'SCOP\tOnyinye_Ndidi\tLeicester\n', 'SCOT\tOnyinye_Ndidi\t95\n', 'SCOP\tDemarai_Gray\tLeicester_City\n', 'SCOT\tDemarai_Gray\tTrước khi trận đấu khép lại\n']


'21495046.txt'

In [ ]:
for line in lines:
    line = line.strip()
    tokens = line.split("\t")
    if len(tokens) == 2:
        entities[tokens[0]].add(tokens[1])
    else:
        relations[tokens[0]].add((tokens[1], tokens[2]))
    
print(*entities)
print()
print(*relations)

CLU PSC TSC SCO

COMP SCOP SCOT


In [ ]:
if len(entities.get('CLU', [])) == 2:
    team1, team2 = tuple(entities['CLU'])
# team1, team2

In [ ]:
import json
from collections import defaultdict
from os import listdir
from os.path import isfile, join

# find longest common subsequence to remove team_name inconsistency
def lcs(X, Y): 
    # find the length of the strings 
    m = len(X) 
    n = len(Y) 
  
    # declaring the array for storing the dp values 
    L = [[None]*(n + 1) for i in range(m + 1)] 
  
    """Following steps build L[m + 1][n + 1] in bottom up fashion 
    Note: L[i][j] contains length of LCS of X[0..i-1] 
    and Y[0..j-1]"""
    for i in range(m + 1): 
        for j in range(n + 1): 
            if i == 0 or j == 0 : 
                L[i][j] = 0
            elif X[i-1] == Y[j-1]: 
                L[i][j] = L[i-1][j-1]+1
            else: 
                L[i][j] = max(L[i-1][j], L[i][j-1]) 
  
    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1] 
    return L[m][n] 
# end of function lcs 


def merge_team_time(player_time, player_team, team_only=False, time_only=False):
    """
    team_only: only keep information in which team is available
    time_only: only keep information in which time is available
    """
    pt = defaultdict(list)
    players = set()
    time_players, team_players = set(), set()
    for player, time in player_time:
        pt[player].append(time)
        players.add(player)
        time_players.add(player)

    pc = defaultdict(str)
    for player, club in player_team:
        pc[player] = club 
        players.add(player)
        team_players.add(player)

    # merge information from the above two dictionaries
    result_list = []
    if team_only and time_only:
        iterator = time_players.union(team_players)
    elif team_only:
        iterator = team_players
    elif time_only:
        iterator = time_players
    else:
        iterator = players

    for player in iterator:
        team = pc[player]
        times = pt[player]
        
        if not times:
            # if time info is not available
            result_list.append({
                "player_name": player.replace('_', ' '),
                "team": team.replace('_', ' '),
                "time": ""
            })
        else:
            for time in times:
                result_list.append({
                    "player_name": player.replace('_', ' '),
                    "team": team.replace('_', ' '),
                    "time": time
                })

    return result_list


def postprocess(prediction_dir, output_file='test_result.jsonl', time_only=False, team_only=False):
    test_result = []
    files = [f for f in listdir(prediction_dir) if isfile(join(prediction_dir, f))]

    for file in files:
        entities = defaultdict(set)
        relations = defaultdict(set)
        test_id = file.split('/')[-1].split('.')[0]

        with open(join(prediction_dir, file), 'r') as f:
            lines = f.readlines()
        #get entities and realations
        for line in lines:
            line = line.strip()
            tokens = line.split('\t')

            if len(tokens) == 2:
                entities[tokens[0]].add(tokens[1])
            else:
                relations[tokens[0]].add((tokens[1], tokens[2]))

        match_summary = {}

        team1, team2 = "", "" 


        # 1. check SCOP relation => prioritize SCOP
        for _, club in relations.get('SCOP', []):
            if not club.isdigit():

                if lcs(team1, club) == len(team1):
                    team1 = club

                elif lcs(team2, club) == len(team2):
                    team2 = club

            # Ensure team1 <> team2
            if team1 != '' and lcs(team1, team2) == len(team1):
                team1 = ''

            elif team2 != '' and lcs(team1, team2) == len(team2):
                team2 = ''


        #     print(f"{test_id} - team1: {team1} - team2: {team2}")

        # 2.check COMP relation
        for club1, club2 in relations.get('COMP', []):
            if not club1.isdigit() and not club2.isdigit():
                
                # 3 cases:
                # - 2 teams are empty => unpack normally
                # - team 1 is empty => team2 is also empty => 1st case
                # - team 1 is not empty 

                if team1 != '':
                    if lcs(team1, club1) == len(club1):
                        team2 = club2
                    else: # team1 = club2 => team2 = club1 | team1 <> club1 & team1 <> club2 => team2 = club1
                        team2 = club1

                else:   
                    team1, team2 = club1, club2
            
            # Ensure team1 <> team2
            if team1 != '' and lcs(team1, team2) == len(team1):
                team1 = ''

            elif team2 != '' and lcs(team1, team2) == len(team2):
                team2 = ''

        #     print(f"{test_id} - team1: {team1} - team2: {team2}")

        # 3.check SCOC relation
        for club, _ in relations.get('SCOC', []):
            if not club.isdigit():
                if lcs(team1, club) == len(team1):
                    team1 = club

                elif lcs(team2, club) == len(team2):
                    team2 = club

            # Ensure team1 <> team2
            if team1 != '' and lcs(team1, team2) == len(team1):
                team1 = ''

            elif team2 != '' and lcs(team1, team2) == len(team2):
                team2 = ''
  
        # print(f"{test_id} - team1: {team1} - team2: {team2}")


        # 4. check CARP relation
        for _, club in relations.get('CARP', []):
            if not club.isdigit():
    
                if lcs(team1, club) == len(team1):
                    team1 = club

                elif lcs(team2, club) == len(team2):
                    team2 = club

            # Ensure team1 <> team2
            if team1 != '' and lcs(team1, team2) == len(team1):
                team1 = ''

            elif team2 != '' and lcs(team1, team2) == len(team2):
                team2 = ''
        
        #     print(f"{test_id} - team1: {team1.replace('_', ' ')} - team2: {team2.replace('_', ' ')}")
        # print(f"{test_id} - team1: {team1} - team2: {team2} ")

        # 5.check CLU entity

        for club in set(entities['CLU']):  
            if not club.isdigit():

                if lcs(team1, club) == len(team1):
                    team1 = club
                
                elif lcs(team2, club) == len(team2):
                    team2 = club

            # Ensure team1 <> team2
            if team1 != '' and lcs(team1, team2) == len(team1):
                team1 = ''

            elif team2 != '' and lcs(team1, team2) == len(team2):
                team2 = ''


        #     print(f"{test_id} - team1: {team1.replace('_', ' ')} - team2: {team2.replace('_', ' ')}")

    
        match_summary["players"] = {
            "team1": team1.replace('_', ' '),
            "team2": team2.replace('_', ' ')
        }


        # Get a list of all players name

        players = entities.get('PSC', set())
        
        for player in entities.get('PCA', set()):
            players.add(player)

        for player in entities.get('PSO', set()):
            players.add(player)

        for player, _ in relations.get('SCOT', []):
            players.add(player)

        for player, _ in relations.get('SCOP', []):
            players.add(player)

        for player, _ in relations.get('CARP', []):
            players.add(player)

        for player, _ in relations.get('CART', []):
            players.add(player)

        for player_in, player_out in relations.get('SUBP', []):
            players.add(player_in)
            players.add(player_out)


        # print(f"{test_id} - players: {players}")

        # By default, set score1, score2 = 0, 0 
        team_score_mapping = {team1: 0, team2: 0}

        # Extract all scores possible

        # 1. SCOP relation
        # group player by team => num of players = team score
        for player, team in relations.get('SCOP', []):

            # FUNC: to handle cases where CLU in SCOP is short ver of team name
            if lcs(team, team1) == len(team): # if team1 contains team
                team = team1
            elif lcs(team, team2) == len(team):
                team = team2

            if team_score_mapping.get(team) is not None:
                team_score_mapping[team] += 1
        print('team_score_mapping 0', team_score_mapping)
        # 2. SCOC relation
        for team, score in relations.get('SCOC', []):

            # FUNC: to handle cases where CLU in SCOC is short ver of team name
            if lcs(team, team1) == len(team): # if team1 contains team
                team = team1
            elif lcs(team, team2) == len(team):
                team = team2

            if team_score_mapping.get(team) == 0 and score.isdigit():
                team_score_mapping[team] = score
        print('team_score_mapping 1', team_score_mapping)
        # 3. SCO 
        # only handle DRAW case
        scores = entities.get('SCO', set())
        print('scores', scores)
        try:
            if len(scores) == 1 and (team_score_mapping.get(team1) != 0 and team_score_mapping.get(team2) != 0): # the two teams draw
                team_score_mapping[team1] += 1
                team_score_mapping[team2] += 1

            elif len(scores) == 2 and (team_score_mapping.get(team1) == 0 and team_score_mapping.get(team2) == 0): 
                # handle (0, 0) scoreboard. If there're only 2 scores, assign randomly
                team_score_mapping[team1] = scores.pop()
                team_score_mapping[team2] = scores.pop()
        except:
            pass
            
        print('scores 1', team_score_mapping[team1])
        print('scores 2', team_score_mapping[team2])
        match_summary["score_board"] = {
            "score1": str(team_score_mapping[team1]),
            "score2": str(team_score_mapping[team2]) 
        }

        #     print(f"{test_id} - score: {team_score_mapping}")

        # Get score list

        score_list = merge_team_time(relations.get('SCOT', []), relations.get('SCOP', []),
                            time_only=time_only, team_only=team_only)
        match_summary["score_list"] = score_list
        
        # print(f"{test_id} - score: {score_list}")

        card_list = merge_team_time(relations.get('CART', []), relations.get('CARP', []),
                                time_only=time_only, team_only=team_only)
        match_summary["card_list"] = card_list


        
        # print(f"{test_id} - score: {card_list}")

        # get substitution list
        sub_list = []
        sub_time = defaultdict(str)

        for player_in, time in relations.get('SUBT', []):
            sub_time[player_in] = time

        for player_in, player_out in relations.get('SUBP', []):
            sub_info = { 
                "player_in": player_in.replace('_', ' '),
                "player_out": player_out.replace('_', ' '),
                "time": sub_time[player_in]
            }
            sub_list.append(sub_info)

        match_summary["substitution_list"] = sub_list

        # print(f"{test_id} - score: {sub_list}")

        test_result.append({
            "test_id": test_id, 
            "match_summary": match_summary
        })

    with open(output_file, 'w') as f:
        for article in test_result:
            json.dump(article, f, ensure_ascii=False)
            f.write('\n')
           

if __name__ == "__main__":
    postprocess('predictions', time_only=False)

team_score_mapping 0 {'Chelsea': 3, 'Arsenal': 1}
team_score_mapping 1 {'Chelsea': 3, 'Arsenal': 1}
scores {'1'}
scores 1 4
scores 2 2
team_score_mapping 0 {'Swansea': 1, 'Man_City': 1}
team_score_mapping 1 {'Swansea': 1, 'Man_City': 1}
scores {'1', '2'}
scores 1 1
scores 2 1
team_score_mapping 0 {'Hull_City': 2, 'Liverpool': 0}
team_score_mapping 1 {'Hull_City': 2, 'Liverpool': 0}
scores {'0', '2'}
scores 1 2
scores 2 0
team_score_mapping 0 {'U23 Việt_Nam': 2, 'U23 Malaysia': 0}
team_score_mapping 1 {'U23 Việt_Nam': 2, 'U23 Malaysia': 0}
scores {'3', '0'}
scores 1 2
scores 2 0
team_score_mapping 0 {'Sampdoria': 1, 'AC Milan': 0}
team_score_mapping 1 {'Sampdoria': 1, 'AC Milan': 0}
scores {'1', '0'}
scores 1 1
scores 2 0
team_score_mapping 0 {'Atletico': 1, 'Leganes': 0}
team_score_mapping 1 {'Atletico': 1, 'Leganes': 0}
scores {'0', '2'}
scores 1 1
scores 2 0
team_score_mapping 0 {'Câu_lạc_bộ Hà_Nội': 2, 'SHB Đà_Nẵng': 0}
team_score_mapping 1 {'Câu_lạc_bộ Hà_Nội': 2, 'SHB Đà_Nẵng': 0}

# Convert data


In [ ]:
import json
import itertools
data = []

data_type = "train"
with open(f"{data_type}.jsonl", "r") as f:
    for line in f:
        # print(line)
        data.append(json.loads(line))
print("Number of examples:", len(data))

Number of examples: 560


In [ ]:
!pip install vncorenlp

     |████████████████████████████████| 2.7MB 9.4MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp37-none-any.whl size=2645936 sha256=acfc8c8a19a386e4a8aa3a971bebacc9bcbfa19637265876cd42555d8cf8c2f5
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp


In [ ]:
class Entity():
    global_id = 1
    def __init__(self, ner_type, start_pos, end_pos, text):
        self.id = f"T-{Entity.global_id}"
        self.ner_type = ner_type
        self.start_pos = start_pos
        self.end_pos = end_pos
        self.text = text
        Entity.global_id += 1
        
    def __repr__(self):
        return f"{self.id}\t{self.ner_type} {self.start_pos} {self.end_pos}\t{self.text}"
        
class Relation():
    global_id = 1
    def __init__(self, relation_type, *args):
        self.id = f"R-{Relation.global_id}"
        self.relation_type = relation_type
        self.args = args
        Relation.global_id += 1
        
    def __repr__(self):
        res = f"{self.id}\t{self.relation_type}"
        for args_id, entity in enumerate(self.args):
            res += f" Arg{args_id + 1}:{entity.id}"
        return res
    
def segmentize(text):
    text = segmentor.tokenize(text.strip())
    return " ".join(" ".join(x) for x in text)

In [ ]:
from bs4 import BeautifulSoup

def segmentize(text):
    text = segmentor.tokenize(text.strip())
    return " ".join(" ".join(x) for x in text)

def parse_paragraph(data):
    event_texts = {0: ""}
    event_paddings = {0: 0}
    event_ids = []
    e_id = None
    content = ""
    
    for html in data["html_annotation"]:
        for e in BeautifulSoup(html, "html.parser"):
            try:
                e_id = int(e["event_id"])
                event_ids.append(e_id)
                event_texts[e_id] = ""
            except: pass
            if e_id is None:
                continue
            event_texts[e_id] += segmentize(e.text.replace("-", " - ")) + " "
            
    # event_ids.sort()
    
    for i, event in enumerate(event_ids):
        cur_event_ids = event_ids[i]
        pre_event_ids = event_ids[i - 1] if i != 0 else 0
        event_paddings[cur_event_ids] = event_paddings[pre_event_ids] + len(event_texts[pre_event_ids])
        content += event_texts[cur_event_ids]
    
    event_texts["0"] = content
    title = segmentize(data["original_doc"]["_source"]["description"])
    return {
        "title": title,
        "content": content,
        "event_texts": event_texts,
        "event_paddings": event_paddings,
    }

In [ ]:
def is_match(team1, team2):
    team1 = team1.strip()
    team2 = team2.strip()
    return team1 in team2 or team2 in team1

In [ ]:
import itertools
def full_text_token(text, start, passage):
    end = start + len(text)
    while end < len(passage) and passage[end] != " ":
        end += 1
    while start > 0 and passage[start - 1] != " ":
        start -= 1
    text = passage[start : end]
    return start, text

In [ ]:
def _parse_entities_other(ner_type, text, ref_event_ids):
    def parse_entity_other(text, eid):
        if text == "":
            return Entity(ner_type, -1, -1, text)
        
        text = segmentize(text)
        pos = event_texts[eid].lower().find(text.lower())
        if pos != -1:
            pos, text = full_text_token(text, pos, event_texts[eid])
            start_pos = pos + event_paddings[eid]
            end_pos = start_pos + len(text)
            return Entity(ner_type, start_pos, end_pos, text)

        edited_texts = []
        if ner_type in ['PSC', 'PCA', 'PSI', 'PSO'] and len(text.split()) != 1:
            edited_texts = text.split()
        elif ner_type in ['TSC', 'TCA', 'TSI'] and "'" in text:
            text = text.replace("'", "")
            edited_texts = ["phút thứ " + text, "phút " + text] 
        for text in edited_texts:
            edited_ner = parse_entity_other(text, eid)
            if edited_ner.start_pos != edited_ner.end_pos:
                return edited_ner

        return Entity(ner_type, -1, -1, text)

    res = []
    for eid in ref_event_ids.split(","):
        entity = parse_entity_other(text, int(eid))
        if entity.start_pos == entity.end_pos or not entity.text or entity.start_pos in [ent.start_pos for ent in res]:
            continue
        res.append(entity)
        # print(res)
    return res

import re
def _parse_entities_score(ner_type, score1, score2, ref_event_ids):
    score1 = score1.replace(' ', '')
    score2 = score2.replace(' ', '')
    score1 = re.sub('\D', '', score1)
    score2 = re.sub('\D', '', score2)
    def parse_entity_single_score(eid, score, start, end):
        pos = event_texts[eid][start : end].find(score)
        if pos != -1:
            pos, score = full_text_token(score, start + pos, event_texts[eid])
            start_pos = event_paddings[eid] + pos
            end_pos = start_pos + len(score)
            
            return Entity(ner_type, start_pos, end_pos, score)
        return Entity(ner_type, -1, -1, "")
    
    def parse_entity_couple_score(eid, score1, score2):
        edited_scores = [score1 + " - " + score2, score2 + " - " + score1]      
        for score in edited_scores:
            score = segmentize(score)
            score_ner = parse_entity_single_score(eid, score, 0, -1)
            if score_ner.start_pos != score_ner.end_pos:
                return score_ner
        return Entity(ner_type, -1, -1, "")

    
    score1_entities, score2_entities = [], []
    for eid in ref_event_ids.split(","):
        eid = int(eid)
        entity = parse_entity_couple_score(eid, score1, score2)
        if entity.start_pos == entity.end_pos:
            score1_entity = parse_entity_single_score(eid, score1, 0, -1)
            score2_entity = parse_entity_single_score(eid, score2, 0, -1)
        else:
            entity.start_pos -= event_paddings[eid]
            entity.end_pos -= event_paddings[eid]
            score1_entity = parse_entity_single_score(eid, score1, entity.start_pos, entity.end_pos)
            score2_entity = parse_entity_single_score(eid, score2, entity.start_pos, entity.end_pos)
           
        if score1_entity.start_pos != score1_entity.end_pos:
            score1_entities.append(score1_entity)
        if score2_entity.start_pos != score2_entity.end_pos:
            score2_entities.append(score2_entity)
        
    return score1_entities, score2_entities

MAX_DISTANCE = 400
def _parse_relations(rel_type, args):
    global MAX_DISTANCE
    res = []
    for entities in itertools.product(*args):
        head_entity, end_entity = entities
        if abs(head_entity.start_pos - end_entity.start_pos) <= MAX_DISTANCE:
        # if abs(head_entity.start_pos - end_entity.start_pos) <= MAX_DISTANCE or rel_type in ['SCOC', 'SCOP', 'CARP']:
            res.append(Relation(rel_type, *entities))
    return res

In [ ]:
def parse_entity_relation(data, event_texts, event_paddings):
    entities, relations = [], []
    id = data["train_id"]
    summary = data["match_summary"]
    team = summary["players"]
    score_board = summary["score_board"]
    score_list = summary["score_list"]
    card_list = summary["card_list"]
    subst_list = summary["substitution_list"]
    
    # Yield teams' name
    team1_entities = _parse_entities_other("CLU", team["team1"], team["ref_event_ids"])
    team2_entities = _parse_entities_other("CLU", team["team2"], team["ref_event_ids"])
    entities.extend(team1_entities + team2_entities)
    relations.extend(_parse_relations("COMP", [team1_entities, team2_entities]))
    relations.extend(_parse_relations("COMP", [team2_entities, team1_entities]))
    
    # Yield scores
    score1_entities, score2_entities = _parse_entities_score("SCO", score_board["score1"], score_board["score2"], score_board["ref_event_ids"])
    entities.extend(score1_entities + score2_entities)
    relations.extend(_parse_relations("SCOC", [team1_entities, score1_entities]))
    relations.extend(_parse_relations("SCOC", [team2_entities, score2_entities]))
    score_team_entities = team1_entities + team1_entities
    print('team1_entities', team1_entities)
    print('team2_entities', team2_entities)
    # Yield list
    for info in score_list:
        player_entities = _parse_entities_other("PSC", info["player_name"], info["ref_event_ids"])
        time_entities = _parse_entities_other("TSC", info["time"], info["ref_event_ids"])
        team_entities = _parse_entities_other("CLU", info["team"], info["ref_event_ids"])
        #HUNGHC
        temp = list(filter(lambda x: x.start_pos not in [e.start_pos for e in score_team_entities], team_entities))
        score_team_entities += temp[::]
        print('score_team_entities', score_team_entities)
        entities.extend(player_entities + time_entities + temp)
        def func(x):
            pos = [e.start_pos for e in score_team_entities].index(x.start_pos)
            if pos == -1:
                return x
            return score_team_entities[pos]
        new_teams = list(map(func, team_entities))
        print('new_teams', new_teams)
        relations.extend(_parse_relations("SCOP", [player_entities, new_teams]))
        relations.extend(_parse_relations("SCOT", [player_entities, time_entities]))       
    teams = score_team_entities + team1_entities + team1_entities
    for info in card_list:
        player_entities = _parse_entities_other("PCA", info["player_name"], info["ref_event_ids"])
        time_entities = _parse_entities_other("TCA", info["time"], info["ref_event_ids"])
        team_entities = _parse_entities_other("CLU", info["team"], info["ref_event_ids"])
        temp = list(filter(lambda x: x.start_pos not in [e.start_pos for e in teams], team_entities))
        teams += temp[::]
        print('teams', teams)
        entities.extend(player_entities + time_entities + temp)
        def func(x):
            pos = [e.start_pos for e in teams].index(x.start_pos)
            if pos == -1:
                return x
            return teams[pos]
        new_teams = list(map(func, team_entities))
        relations.extend(_parse_relations("CARP", [player_entities, new_teams]))
        relations.extend(_parse_relations("CART", [player_entities, time_entities]))   
        
    for info in subst_list:
        playerin_entities = _parse_entities_other("PSI", info["player_in"], info["ref_event_ids"])
        playerout_entities = _parse_entities_other("PSO", info["player_out"], info["ref_event_ids"])
        time_entities = [] if "time" not in info else _parse_entities_other("TSI", info["time"], info["ref_event_ids"])
        entities.extend(playerin_entities + playerout_entities + time_entities)
        relations.extend(_parse_relations("SUBP", [playerin_entities, playerout_entities]))
        relations.extend(_parse_relations("SUBT", [playerin_entities, time_entities]))
    
    return entities, relations

In [ ]:
data[0]["html_annotation"]

['<span class="tag match_info" data=":match_info:" event_id="1">HLV Julen Lopetegui trao cơ hội cho thủ thành trẻ Kepa Arrizabalaga thay thế vị trí David de Gea trong khung thành của Tây Ban Nha trong cuộc tiếp đón Costa Rica trên sân La Rosaleda.</span>',
 '<span>Với lợi thế sân nhà cùng chất lượng vượt trội, Tây Ban Nha không khó để chiếm lĩnh thế trận trước các đội khách đến từ trung Mỹ. Thống kê ở hiệp thi đấu thứ nhất, La Roja cầm bóng tới 76% và tung ra 13 pha dứt điểm về phía khung thành đội khách.</span>',
 '<span class="tag goal_info" data=":goal_info:" event_id="2">Chỉ 6 phút sau tiếng còi khai cuộc, đội chủ nhà đã có bàn mở tỷ số nhờ công của Jordi Alba. Tiền vệ đang thi đấu cho Barcelona chớp cơ hội rất nhanh sau cú cang ngang của David Silva từ bên phía cánh phải để hạ gục thủ thành Danny Carvajal (Costa Rica).</span>',
 '<span class="tag goal_info" data=":goal_info:" event_id="3">Có được bàn mở tỷ số, Tây Ban Nha liên tiếp dồn ép đối thủ và bàn thắng thứ 2 đến với họ như 

In [ ]:
from vncorenlp import VnCoreNLP
segmentor = VnCoreNLP("VnCoreNLP-1.1.1/VnCoreNLP-1.1.1.jar", annotators="wseg")

In [ ]:
import json
import itertools
data = []

data_type = "dev"
with open(f"{data_type}.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))
print("Number of examples:", len(data))

Number of examples: 141


In [ ]:
sample_idx = 1
print(data[sample_idx])
paragraph = parse_paragraph(data[sample_idx])

content, event_texts, event_paddings = paragraph["content"], paragraph["event_texts"], paragraph["event_paddings"]
entities, relations = parse_entity_relation(data[sample_idx], event_texts, event_paddings)

print(*entities, sep="\n")
print('===')
print(*relations, sep="\n")

{'train_id': '22795492', 'original_doc': {'_id': '22795492', '_source': {'body': [{'content': 'Đối thủ U23 Timor Leste được đánh giá yếu hơn nên đây là cơ hội để thầy trò HLV Nguyễn Hữu Thắng ghi thật nhiều bàn thắng để có lợi thế về hiệu số so với các đối thủ cạnh tranh.', 'text': 'Đối thủ U23 Timor Leste được đánh giá yếu hơn nên đây là cơ hội để thầy trò HLV Nguyễn Hữu Thắng ghi thật nhiều bàn thắng để có lợi thế về hiệu số so với các đối thủ cạnh tranh.', 'type': 'text'}, {'content': '<strong>Đội hình ra sân:</strong>', 'text': 'Đội hình ra sân:', 'type': 'text'}, {'content': '<strong>U23 Việt Nam: </strong>Tiến Dũng, Văn Thanh, Văn Khánh, Bùi Tiến Dũng, Văn Hậu, Văn Toàn, Duy Mạnh, Tuấn Anh, Công Phượng, Hồng Duy, Đức Chinh.', 'text': 'U23 Việt Nam: Tiến Dũng, Văn Thanh, Văn Khánh, Bùi Tiến Dũng, Văn Hậu, Văn Toàn, Duy Mạnh, Tuấn Anh, Công Phượng, Hồng Duy, Đức Chinh.', 'type': 'text'}, {'content': '<strong>U23 Đông Timor:</strong> Augusto, Oliveira, Sabas, Mendonca, Viegas, Trind

In [ ]:
from tqdm import tqdm
import json
import itertools
data = []

data_type = "train"
with open(f"{data_type}.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))
        # print(line)
print("Number of examples:", len(data))
if data_type == "train":
    data[83]["match_summary"]["score_list"][1]["ref_event_ids"] = "4"
    data[361]["match_summary"]["score_list"][0]["ref_event_ids"] = "4"
    data[346]["match_summary"]["card_list"] = []
    data[375]["match_summary"]["score_list"][1]["ref_event_ids"] = "3"

for x in tqdm(data):
    file_id = x["train_id"]
    paragraph = parse_paragraph(x)
    content, event_texts, event_paddings = paragraph["content"], paragraph["event_texts"], paragraph["event_paddings"]
    entities, relations = parse_entity_relation(x, event_texts, event_paddings)
    
    with open(f"./{data_type}/{file_id}.txt", "w") as f:
        f.write(paragraph["content"])
    
    with open(f"./{data_type}/{file_id}.ann", "w") as f:
        for entity in entities:
            f.write(str(entity) + "\n")
        for relation in relations:
            f.write(str(relation) + "\n")



  0%|          | 0/560 [00:00<?, ?it/s]

Number of examples: 560
team1_entities [T-104854	CLU 63 69	Alaves]
team2_entities [T-104855	CLU 96 107	Real_Madrid]




  0%|          | 1/560 [00:00<02:45,  3.37it/s]

score_team_entities [T-104854	CLU 63 69	Alaves, T-104854	CLU 63 69	Alaves, T-104863	CLU 0 6	Alaves]
new_teams [T-104863	CLU 0 6	Alaves]




  0%|          | 2/560 [00:00<02:37,  3.54it/s]

...

team1_entities [T-118108	CLU 351 355	HAGL]
team2_entities [T-118109	CLU 381 399	U18 Mito_Hollyhock]
score_team_entities [T-118108	CLU 351 355	HAGL, T-118108	CLU 351 355	HAGL, T-118115	CLU 1462 1466	HAGL]
new_teams [T-118115	CLU 1462 1466	HAGL]
score_team_entities [T-118108	CLU 351 355	HAGL, T-118108	CLU 351 355	HAGL, T-118115	CLU 1462 1466	HAGL, T-118118	CLU 1706 1710	HAGL]
new_teams [T-118118	CLU 1706 1710	HAGL]
team1_entities [T-118119	CLU 306 310	Real]
team2_entities [T-118120	CLU 209 214	Barca]
score_team_entities [T-118119	CLU 306 310	Real, T-118119	CLU 306 310	Real, T-118126	CLU 775 779	Real]
new_teams [T-118126	CLU 775 779	Real]
score_team_entities [T-118119	CLU 306 310	Real, T-118119	CLU 306 310	Real, T-118126	CLU 775 779	Real, T-118129	CLU 1234 1239	Barca]
new_teams [T-118129	CLU 1234 1239	Barca]
score_team_entities [T-118119	CLU 306 310	Real, T-118119	CLU 306 310	Real, T-118126	CLU 775 779	Real, T-118129	CLU 1234 1239	Barca, T-118132	CLU 1909 1913	Real]
new_teams [T-118132	CL



 99%|█████████▉| 557/560 [01:27<00:00,  6.24it/s]

100%|█████████▉| 558/560 [01:27<00:00,  6.75it/s]

team1_entities [T-118145	CLU 52 60	Man_City]
team2_entities [T-118146	CLU 0 7	Everton]
score_team_entities [T-118145	CLU 52 60	Man_City, T-118145	CLU 52 60	Man_City, T-118154	CLU 0 7	Everton, T-118155	CLU 1909 1916	Everton]
new_teams [T-118154	CLU 0 7	Everton, T-118155	CLU 1909 1916	Everton]
score_team_entities [T-118145	CLU 52 60	Man_City, T-118145	CLU 52 60	Man_City, T-118154	CLU 0 7	Everton, T-118155	CLU 1909 1916	Everton, T-118160	CLU 347 355	Man_City]
new_teams [T-118160	CLU 347 355	Man_City]
teams [T-118145	CLU 52 60	Man_City, T-118145	CLU 52 60	Man_City, T-118154	CLU 0 7	Everton, T-118155	CLU 1909 1916	Everton, T-118160	CLU 347 355	Man_City, T-118145	CLU 52 60	Man_City, T-118145	CLU 52 60	Man_City, T-118166	CLU 276 284	Man_City, T-118167	CLU 2484 2492	Man_City]
teams [T-118145	CLU 52 60	Man_City, T-118145	CLU 52 60	Man_City, T-118154	CLU 0 7	Everton, T-118155	CLU 1909 1916	Everton, T-118160	CLU 347 355	Man_City, T-118145	CLU 52 60	Man_City, T-118145	CLU 52 60	Man_City, T-118166	



100%|██████████| 560/560 [01:27<00:00,  6.40it/s]

team1_entities [T-118199	CLU 69 72	PSG]
team2_entities [T-118200	CLU 77 83	Monaco]
score_team_entities [T-118199	CLU 69 72	PSG, T-118199	CLU 69 72	PSG]
new_teams []
score_team_entities [T-118199	CLU 69 72	PSG, T-118199	CLU 69 72	PSG, T-118209	CLU 1411 1414	PSG]
new_teams [T-118209	CLU 1411 1414	PSG]
score_team_entities [T-118199	CLU 69 72	PSG, T-118199	CLU 69 72	PSG, T-118209	CLU 1411 1414	PSG, T-118212	CLU 2139 2142	PSG]
new_teams [T-118212	CLU 2139 2142	PSG]
score_team_entities [T-118199	CLU 69 72	PSG, T-118199	CLU 69 72	PSG, T-118209	CLU 1411 1414	PSG, T-118212	CLU 2139 2142	PSG, T-118215	CLU 2831 2834	PSG]
new_teams [T-118215	CLU 2831 2834	PSG]
team1_entities [T-118216	CLU 0 10	Sài_Gòn FC]
team2_entities []
score_team_entities [T-118216	CLU 0 10	Sài_Gòn FC, T-118216	CLU 0 10	Sài_Gòn FC]
new_teams []
score_team_entities [T-118216	CLU 0 10	Sài_Gòn FC, T-118216	CLU 0 10	Sài_Gòn FC, T-118226	CLU 341 351	Sài_Gòn FC]
new_teams [T-118226	CLU 341 351	Sài_Gòn FC]


In [ ]:
import json
import itertools
data = []

data_type = "test"
with open(f"{data_type}.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))
print("Number of examples:", len(data))

Number of examples: 211


In [ ]:
for x in tqdm(data):
    file_id = x["test_id"]
    content = " ".join(segmentize(body["text"].replace("-", " - ")) for body in x["original_doc"]["_source"]["body"])
    
    with open(f"./{data_type}/{file_id}.txt", "w") as f:
        f.write(content)
    with open(f"./{data_type}/{file_id}.ann", "w") as f:
        for entity in entities:
            f.write("")
        for relation in relations:
            f.write("")



  0%|          | 0/211 [00:00<?, ?it/s]

  1%|          | 2/211 [00:00<00:14, 14.47it/s]

  2%|▏         | 4/211 [00:00<00:13, 15.48it/s]

  3%|▎         | 7/211 [00:00<00:11, 17.44it/s]

  5%|▍         | 10/211 [00:00<00:11, 17.79it/s]

  6%|▌         | 12/211 [00:00<00:12, 16.05it/s]

  7%|▋         | 14/211 [00:00<00:11, 16.93it/s]

  8%|▊         | 16/211 [00:00<00:11, 17.59it/s]

  9%|▊         | 18/211 [00:01<00:11, 16.35it/s]

 10%|█         | 22/211 [00:01<00:10, 18.74it/s]

 12%|█▏        | 25/211 [00:01<00:09, 19.85it/s]

 13%|█▎        | 28/211 [00:01<00:11, 15.65it/s]

 14%|█▍        | 30/211 [00:01<00:10, 16.60it/s]

 16%|█▌        | 33/211 [00:01<00:09, 17.90it/s]

 17%|█▋        | 35/211 [00:02<00:12, 14.15it/s]

 18%|█▊        | 38/211 [00:02<00:10, 15.85it/s]

 19%|█▉        | 41/211 [00:02<00:10, 16.81it/s]

 20%|██        | 43/211 [00:02<00:12, 13.72it/s]

 21%|██▏       | 45/211 [00:02<00:10, 15.14it/s]

 23%|██▎       | 48/211 [00:02<00:09, 16.75it/s]

 24%|██▎  

In [ ]:
import json
import itertools
data = []

data_type = "dev"
with open(f"{data_type}.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))
print("Number of examples:", len(data))

Number of examples: 141


In [ ]:
data_type = "dev"
if data_type == "train":
    data[346]["match_summary"]["card_list"] = []
    data[375]["match_summary"]["score_list"][1]["ref_event_ids"] = "3"
    
for x in tqdm(data):
    file_id = x["train_id"]
    paragraph = parse_paragraph(x)
    content, event_texts, event_paddings = paragraph["content"], paragraph["event_texts"], paragraph["event_paddings"]
    entities, relations = parse_entity_relation(x, event_texts, event_paddings)
    
    with open(f"./{data_type}/{file_id}.txt", "w") as f:
        f.write(paragraph["content"])
    
    with open(f"./{data_type}/{file_id}.ann", "w") as f:
        for entity in entities:
            f.write(str(entity) + "\n")
        for relation in relations:
            f.write(str(relation) + "\n")



  0%|          | 0/141 [00:00<?, ?it/s]

  1%|          | 1/141 [00:00<00:15,  8.96it/s]

team1_entities [T-118227	CLU 119 130	Tây_Ban_Nha]
team2_entities [T-118228	CLU 151 161	Costa_Rica]
score_team_entities [T-118227	CLU 119 130	Tây_Ban_Nha, T-118227	CLU 119 130	Tây_Ban_Nha]
new_teams []
score_team_entities [T-118227	CLU 119 130	Tây_Ban_Nha, T-118227	CLU 119 130	Tây_Ban_Nha, T-118240	CLU 717 728	Tây_Ban_Nha]
new_teams [T-118240	CLU 717 728	Tây_Ban_Nha]
score_team_entities [T-118227	CLU 119 130	Tây_Ban_Nha, T-118227	CLU 119 130	Tây_Ban_Nha, T-118240	CLU 717 728	Tây_Ban_Nha]
new_teams []
score_team_entities [T-118227	CLU 119 130	Tây_Ban_Nha, T-118227	CLU 119 130	Tây_Ban_Nha, T-118240	CLU 717 728	Tây_Ban_Nha]
new_teams []
score_team_entities [T-118227	CLU 119 130	Tây_Ban_Nha, T-118227	CLU 119 130	Tây_Ban_Nha, T-118240	CLU 717 728	Tây_Ban_Nha]
new_teams []
team1_entities [T-118255	CLU 0 12	U23 Việt_Nam, T-118256	CLU 266 278	U23 Việt_Nam]
team2_entities [T-118258	CLU 147 161	U23 Đông_Timor]
score_team_entities [T-118255	CLU 0 12	U23 Việt_Nam, T-118256	CLU 266 278	U23 Việt_Nam,



  1%|▏         | 2/141 [00:00<00:17,  7.99it/s]

score_team_entities [T-118255	CLU 0 12	U23 Việt_Nam, T-118256	CLU 266 278	U23 Việt_Nam, T-118255	CLU 0 12	U23 Việt_Nam, T-118256	CLU 266 278	U23 Việt_Nam, T-118266	CLU 1501 1513	U23 Việt_Nam, T-118273	CLU 3547 3559	U23 Việt_Nam]
new_teams [T-118273	CLU 3547 3559	U23 Việt_Nam]
team1_entities [T-118286	CLU 0 8	Việt_Nam]
team2_entities [T-118289	CLU 200 208	Pakistan]
score_team_entities [T-118286	CLU 0 8	Việt_Nam, T-118286	CLU 0 8	Việt_Nam, T-118295	CLU 446 454	Việt_Nam]
new_teams [T-118295	CLU 446 454	Việt_Nam]
score_team_entities [T-118286	CLU 0 8	Việt_Nam, T-118286	CLU 0 8	Việt_Nam, T-118295	CLU 446 454	Việt_Nam, T-118298	CLU 974 982	Việt_Nam]
new_teams [T-118298	CLU 974 982	Việt_Nam]
score_team_entities [T-118286	CLU 0 8	Việt_Nam, T-118286	CLU 0 8	Việt_Nam, T-118295	CLU 446 454	Việt_Nam, T-118298	CLU 974 982	Việt_Nam]
new_teams []




  3%|▎         | 4/141 [00:00<00:15,  8.67it/s]

...

score_team_entities [T-121289	CLU 1181 1189	Athletic, T-121290	CLU 693 701	Athletic, T-121289	CLU 1181 1189	Athletic, T-121290	CLU 693 701	Athletic, T-121301	CLU 14 18	Real, T-121302	CLU 204 208	Real, T-121310	CLU 510 514	Real, T-121311	CLU 864 868	Real]
new_teams [T-121310	CLU 510 514	Real, T-121311	CLU 864 868	Real]
team1_entities [T-121312	CLU 61 70	Tottenham]
team2_entities [T-121313	CLU 18 25	Chelsea]
score_team_entities [T-121312	CLU 61 70	Tottenham, T-121312	CLU 61 70	Tottenham, T-121320	CLU 1211 1218	Chelsea]
new_teams [T-121320	CLU 1211 1218	Chelsea]
score_team_entities [T-121312	CLU 61 70	Tottenham, T-121312	CLU 61 70	Tottenham, T-121320	CLU 1211 1218	Chelsea, T-121323	CLU 2360 2369	Tottenham]
new_teams [T-121323	CLU 2360 2369	Tottenham]
score_team_entities [T-121312	CLU 61 70	Tottenham, T-121312	CLU 61 70	Tottenham, T-121320	CLU 1211 1218	Chelsea, T-121323	CLU 2360 2369	Tottenham, T-121326	CLU 2751 2758	Chelsea]
new_teams [T-121326	CLU 2751 2758	Chelsea]




 99%|█████████▊| 139/141 [00:18<00:00,  6.27it/s]

team1_entities [T-121327	CLU 12 25	Mokpo_City_FC]
team2_entities [T-121328	CLU 68 80	U22 Việt_Nam]
score_team_entities [T-121327	CLU 12 25	Mokpo_City_FC, T-121327	CLU 12 25	Mokpo_City_FC, T-121334	CLU 1735 1747	U22 Việt_Nam]
new_teams [T-121334	CLU 1735 1747	U22 Việt_Nam]
score_team_entities [T-121327	CLU 12 25	Mokpo_City_FC, T-121327	CLU 12 25	Mokpo_City_FC, T-121334	CLU 1735 1747	U22 Việt_Nam]
new_teams [T-121334	CLU 1735 1747	U22 Việt_Nam]
score_team_entities [T-121327	CLU 12 25	Mokpo_City_FC, T-121327	CLU 12 25	Mokpo_City_FC, T-121334	CLU 1735 1747	U22 Việt_Nam, T-121343	CLU 2404 2416	U22 Việt_Nam]
new_teams [T-121343	CLU 2404 2416	U22 Việt_Nam]
score_team_entities [T-121327	CLU 12 25	Mokpo_City_FC, T-121327	CLU 12 25	Mokpo_City_FC, T-121334	CLU 1735 1747	U22 Việt_Nam, T-121343	CLU 2404 2416	U22 Việt_Nam, T-121349	CLU 2656 2668	U22 Việt_Nam]
new_teams [T-121343	CLU 2404 2416	U22 Việt_Nam, T-121349	CLU 2656 2668	U22 Việt_Nam]
score_team_entities [T-121327	CLU 12 25	Mokpo_City_FC, T-



 99%|█████████▉| 140/141 [00:19<00:00,  4.07it/s]

100%|██████████| 141/141 [00:19<00:00,  7.27it/s]

team1_entities [T-121397	CLU 483 491	Việt_Nam]
team2_entities [T-121398	CLU 423 431	Hàn_Quốc]
score_team_entities [T-121397	CLU 483 491	Việt_Nam, T-121397	CLU 483 491	Việt_Nam, T-121411	CLU 31 39	Hàn_Quốc, T-121412	CLU 805 813	Hàn_Quốc]
new_teams [T-121411	CLU 31 39	Hàn_Quốc, T-121412	CLU 805 813	Hàn_Quốc]
score_team_entities [T-121397	CLU 483 491	Việt_Nam, T-121397	CLU 483 491	Việt_Nam, T-121411	CLU 31 39	Hàn_Quốc, T-121412	CLU 805 813	Hàn_Quốc, T-121418	CLU 1357 1365	Hàn_Quốc]
new_teams [T-121411	CLU 31 39	Hàn_Quốc, T-121418	CLU 1357 1365	Hàn_Quốc]
score_team_entities [T-121397	CLU 483 491	Việt_Nam, T-121397	CLU 483 491	Việt_Nam, T-121411	CLU 31 39	Hàn_Quốc, T-121412	CLU 805 813	Hàn_Quốc, T-121418	CLU 1357 1365	Hàn_Quốc, T-121424	CLU 2070 2078	Hàn_Quốc]
new_teams [T-121411	CLU 31 39	Hàn_Quốc, T-121424	CLU 2070 2078	Hàn_Quốc]
score_team_entities [T-121397	CLU 483 491	Việt_Nam, T-121397	CLU 483 491	Việt_Nam, T-121411	CLU 31 39	Hàn_Quốc, T-121412	CLU 805 813	Hàn_Quốc, T-121418	CLU 1357 

In [ ]:
import os
import json
import re
import argparse

from tqdm import tqdm
# from constants import *
from transformers import AutoTokenizer 


def aligment_ann(original, newtext, ann_file, offset):
    # Ensure that uncased tokenizers can also be aligned
    # original = original.lower()
    # newtext = newtext.lower()
    annotation = []
    terms = {}
    ends = {}

    for line in open(ann_file):
        if line.startswith('T'):
            annots = line.rstrip().split("\t", 2)
            typeregion = annots[1].split(" ")
            start = int(typeregion[1]) - offset
            end = int(typeregion[2]) - offset
            if not start in terms:
                terms[start] = []
            if not end in ends:
                ends[end] = []
            if len(annots) == 3:
                terms[start].append(
                    [start, end, annots[0], typeregion[0], annots[2]])
            else:
                terms[start].append([start, end, annots[0], typeregion[0], ""])
            ends[end].append(start)
        else:
            annotation.append(line)
    # print('=================================ann_file===========================', ann_file)
    if ann_file == 'train/21231956.ann':
      print('=================================21231956===========================')
      print('terms', terms)
      print('newtext', newtext)
      print('original', original)
    orgidx = 0
    newidx = 0
    orglen = len(original)
    newlen = len(newtext)

    while orgidx < orglen and newidx < newlen:
        if original[orgidx] == newtext[newidx]:
            orgidx += 1
            newidx += 1
        elif newtext[newidx] == '\n':
            newidx += 1
        elif original[orgidx] == '\n':
            orgidx += 1
        elif newtext[newidx] == ' ':
            newidx += 1
        elif original[orgidx] == ' ':
            orgidx += 1
        elif newtext[newidx] == '\t':
            newidx += 1
        elif original[orgidx] == '\t':
            orgidx += 1
        elif newtext[newidx] == '.':
            # ignore extra "." for stanford
            newidx += 1
        else:
            assert False, "%d\t$%s$\t$%s$" % (
                orgidx, original[orgidx:orgidx + 20], newtext[newidx:newidx + 20])
        if orgidx in terms:
            for l in terms[orgidx]:
                l[0] = newidx
        if orgidx in ends:
            for start in ends[orgidx]:
                for l in terms[start]:
                    if l[1] == orgidx:
                        l[1] = newidx
            del ends[orgidx]

    entities = []
    relations = []
    dict1 = {}
    i = 0
    for ts in terms.values():
        for term in ts:
            if term[4] == "":
                entities.append([term[2], term[3], term[0],
                                 term[1], newtext[term[0]:term[1]]])
            else:
                """
                a = newtext[term[0] : term[1]].replace(" ", "").replace("\n", "")
                b = term[4].replace(" ", "").lower()
                assert newtext[term[0]:term[1]].replace(" ", "").replace('\n', "").replace("&AMP;", "&").replace("&amp;", "&") == \
                    term[4].replace(" ", "").lower(
                ), newtext[term[0]:term[1]] + "<=>" + term[4]
                """
                entities.append([term[2], term[3], term[0], term[1],
                                 newtext[term[0]:term[1]].replace("\n", " ")])
            dict1[term[2]] = i
            i += 1
    if ann_file == 'train/21231956.ann':
      print('entities', entities)
    for rel in annotation:
        rel_id, rel_type, rel_e1, rel_e2 = rel.strip().split()
        rel_e1 = rel_e1[5:]
        rel_e2 = rel_e2[5:]
        relations.append([rel_id, rel_type, rel_e1, rel_e2])
    relations1 = []
    for rel in relations:
        _, rel_type, rel_e1, rel_e2 = rel
        rel_e1_idx = dict1[rel_e1]
        rel_e2_idx = dict1[rel_e2]
        relations1.append([rel_type, rel_e1_idx, rel_e2_idx])
    entities1 = [[ent[1], ent[2], ent[3], ent[4]] for ent in entities]
    if ann_file == 'train/21231956.ann':
      print('entities1', entities1)
    return entities1, relations1


def passage_blocks(txt, window_size, overlap):
    blocks = []
    regions = []
    for i in range(0, len(txt), window_size-overlap):
        b = txt[i:i+window_size]
        blocks.append(b)
        regions.append((i, i+window_size))
    return blocks, regions


def get_block_er(txt, entities, relations, window_size, overlap, tokenizer):
    """
    Get the block level annotation
    Args:
        txt: text to be processed, list of token
        entities: list of (entity_type, start, end,entity)
        relations: list of (relation_type,entity1_idx,entity2_idx)
        window_siez: sliding window size
        overlap: overlap between two adjacent windows
    Returns:
        ber: list of [block，entities, relations]
    """
    blocks, block_range = passage_blocks(txt, window_size, overlap)
    ber = [[[], [], []] for i in range(len(block_range))]
    e_dict = {}
    count = 0
    for i, (s, e) in enumerate(block_range):
        es = []
        for j, (entity_type, start, end, entity_str) in enumerate(entities):
            if start >= s and end <= e:
                nstart, nend = start-s, end-s
                
                if tokenizer.convert_tokens_to_string(blocks[i][nstart:nend]) == entity_str:
                    es.append((entity_type, nstart, nend, entity_str))
                    e_dict[j] = e_dict.get(j, [])+[i]
                else:
                    print(
                        "The entity string and its corresponding index are inconsistent")
        ber[i][0] = blocks[i]
        ber[i][1].extend(es)
    for r, e1i, e2i in relations:
        if e1i not in e_dict or e2i not in e_dict:
            print("Entity lost due to sliding window")
            continue
        i1s, i2s = e_dict[e1i], e_dict[e2i]
        intersec = set.intersection(set(i1s), set(i2s))
        if intersec:
            for i in intersec:
                t1, s1, e1, es1 = entities[e1i][0], entities[e1i][1] - \
                    block_range[i][0], entities[e1i][2] - \
                    block_range[i][0], entities[e1i][3]
                t2, s2, e2, es2 = entities[e2i][0], entities[e2i][1] - \
                    block_range[i][0], entities[e2i][2] - \
                    block_range[i][0], entities[e2i][3]
                ber[i][2].append((r, (t1, s1, e1, es1), (t2, s2, e2, es2)))
        else:
            count += 1
            # print("The two entities of the relationship are not on the same sentence")
    if count > 1:        
        print("Entites of relations in different sentence:", count)
    return ber


def get_question(question_templates, head_entity, relation_type=None, end_entity_type=None):
    """
    Args:
        head_entity: (entity_type,start_idx,end_idx,entity_string) or entity_type
    """
    if relation_type == None:
        question = question_templates['qa_turn1'][head_entity[0]] if isinstance(
            head_entity, tuple) else question_templates['qa_turn1'][head_entity]
    else:
        question = question_templates['qa_turn2'][str(
            (head_entity[0], relation_type, end_entity_type))]
        question = question.replace('XXX', head_entity[3])
    return question


def block2qas(ber, dataset_tag, title="", threshold=1, max_distance=45, id=""):
    """
    Args:
        ber: (block,entities,relations)
        dataset_tag: type of dataset
        title: title corresponding to the passage to which the block belongs
        threshold: only consider relationships where the frequency is greater than or equal to the threshold
        max_distance: used to filter relations by distance from the head entity
    """
    if dataset_tag.lower() == "zalo":
        entities = zalo_entities
        relations = zalo_relations
        idx1s = zalo_idx1
        idx2s = zalo_idx2
        dist = zalo_dist
        question_templates = zalo_question_templates
    elif dataset_tag.lower() == "ace2004":
        entities = ace2004_entities
        relations = ace2004_relations
        idx1s = ace2004_idx1
        idx2s = ace2004_idx2
        dist = ace2004_dist
        question_templates = ace2004_question_templates
    elif dataset_tag.lower() == 'ace2005':
        entities = ace2005_entities
        relations = ace2005_relations
        idx1s = ace2005_idx1
        idx2s = ace2005_idx2
        dist = ace2005_dist
        question_templates = ace2005_question_templates
    else:
        raise Exception("this data set is not yet supported")
    block, ents, relas = ber
    res = {'context': block, 'title': title}

    # QA turn 1
    dict1 = {k: get_question(question_templates, k) for k in entities} # Các câu hỏi turn 1 'CLU' : Liệt kê tất cả các câu lạc bộ
    qat1 = {dict1[k]: [] for k in dict1}
    # Sau khi được list câu hỏi thì trả lời các câu hỏi này
    for en in ents:
        q = dict1[en[0]] # en[0]: head entity 
        qat1[q].append(en)
    # QA turn 2
    if max_distance > 0:
        dict2 = {(rel[1], rel[0], rel[2][0]): [] for rel in relas}
        for rel in relas:
            dict2[(rel[1], rel[0], rel[2][0])].append(rel[2])
        qat2 = []
        ents1 = sorted(ents, key=lambda x: x[1])
        for i, ent1 in enumerate(ents1):
            start = ent1[1]
            qas = {}
            for j, ent2 in enumerate(ents1):
                if i >= j: # HUNGHC ADD
                  continue
                if ent2[1] > start+max_distance:
                    break
                else:
                    head_type, end_type = ent1[0], ent2[0]
                    for rel_type in relations:
                        idx1, idx2 = idx1s[head_type], idx2s[(
                            rel_type, end_type)]
                        if dist[idx1][idx2] >= threshold:
                            k = (ent1, rel_type, end_type)
                            # print('k', k)
                            q = get_question(
                                question_templates, ent1, rel_type, end_type)
                            qas[q] = dict2.get(k, [])
                            # print(qas[q])
            qat2.append({"head_entity": ent1, "qas": qas})
    else:
        dict2 = {(rel[1], rel[0], rel[2][0]): [] for rel in relas}
        for rel in relas:
            dict2[(rel[1], rel[0], rel[2][0])].append(rel[2])
        qat2 = []
        for ent in ents:
            qas = {}
            for rel_type in relations:
                for ent_type in entities:
                    k = (ent, rel_type, ent_type)
                    idx1, idx2 = idx1s[ent[0]], idx2s[(rel_type, ent_type)]
                    if dist[idx1][idx2] >= threshold:
                        q = get_question(question_templates,
                                         ent, rel_type, ent_type)
                        qas[q] = dict2.get(k, [])
            qat2.append({'head_entity': ent, "qas": qas})
    qas = [qat1, qat2]
    res["qa_pairs"] = qas
    return res


def char_to_wordpiece(passage, entities, tokenizer):
    entities1 = []
    tpassage = tokenizer.tokenize(passage)
    #print('entities ================== ', entities)
    for ent in entities:
        ent_type, start, end, ent_str = ent
        #print('start =================', start)
        #print('ent_type', ent_type)
        #print('end', end)
        #print('ent_str', ent_str)
        s = tokenizer.tokenize(passage[:start])
        #print('s', s)
        #print('passage[:start]' , passage[:start+3])
        start1 = len(s)
        #print('start1', start1)
        
        ent_str1 = tokenizer.tokenize(ent_str)
        #print('ent_str1', ent_str1)
        end1 = start1 + len(ent_str1)
        #print('end1', end1)
        ent_str2 = tokenizer.convert_tokens_to_string(ent_str1)
        assert tpassage[start1:end1] == ent_str1, f"Passage: {tpassage[start1 - 10 : end1 + 10]}\nEntity: {ent_str} {ent_str1} {ent_str2}, {tpassage[start1:end1]} != {ent_str1}"
        entities1.append((ent_type, start1, end1, ent_str2))
    return entities1


def process(data_dir, output_dir, tokenizer, is_test, window_size, overlap, dataset_tag, threshold=1, max_distance=45):
    """
    Args:
        data_dir: data directory
        output_dir: output directory
        tokenizer: tokenizer of pretrained model
        is_test: whether it is test data
        window_size: sliding window size
        overlap: overlap between two adjacent windows
        dataset_tag: type of dataset
        threshold： only consider relationships where the frequency is greater than or equal to the threshold
        max_distance: used to filter relations by distance from the head entity
    """
    ann_files = []
    txt_files = []
    data = []
    files = []
    for f in os.listdir(data_dir):
        if f.endswith('.txt'):
            txt_files.append(os.path.join(data_dir, f))
            files.append(f.split(".")[0])
        elif f.endswith('.ann'):
            ann_files.append(os.path.join(data_dir, f))
    ann_files = sorted(ann_files)
    txt_files = sorted(txt_files)
    files = sorted(files)
    
    for fname, ann_path, txt_path in tqdm(zip(files, ann_files, txt_files), total=len(ann_files)):
        # if ann_path != 'train/21231956.ann':
        #   continue
        with open(txt_path, encoding='utf-8') as f:
            raw_txt = f.read()
            txt = [t for t in raw_txt.split('\n') if t.strip()]
        
        # get the title information, the title will be added to all windows of a passage
        title = "" 
        content = txt[0]
        offset = 0
        # offset = raw_txt.index(content)
        ntxt1 = tokenizer.tokenize(content)
        ntxt2 = tokenizer.convert_tokens_to_string(ntxt1)
        entities, relations = aligment_ann(raw_txt[offset:], ntxt2, ann_path, offset)

        # convert entitiy index from char level to wordpiece level
        entities = char_to_wordpiece(ntxt2, entities, tokenizer)
        if is_test:
            data.append({"title": title, "passage": ntxt1, "id": fname,
                         "entities": entities, "relations": relations})
        else:
            block_er = get_block_er(
                ntxt1, entities, relations, window_size, overlap, tokenizer)
            
            for ber in block_er:
                temp = block2qas(ber, dataset_tag,
                                    title, threshold, max_distance, id=ann_path)
                data.append(temp)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    save_path = os.path.join(output_dir, os.path.split(data_dir)[-1]+".json")
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, sort_keys=True, indent=4)

In [ ]:
output_dir = 'dataset'
tokenizer = AutoTokenizer.from_pretrained(argss.pretrained_model_path)
print(zalo_question_templates)
process('test', output_dir, tokenizer, 	True,
        220, 45, argss.dataset_tag, 1, 100)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/211 [00:00<?, ?it/s]

  6%|▌         | 12/211 [00:00<00:01, 115.00it/s]

{'qa_turn1': {'SCO': 'liệt kê tất cả số bàn thắng.', 'PSI': 'liệt kê tất cả cầu thủ thay thế.', 'PSO': 'liệt kê tất cả cầu thủ ra sân.', 'TSI': 'liệt kê tất cả thời gian vào sân.', 'PSC': 'liệt kê tất cả cầu thủ ghi bàn.', 'TSC': 'liệt kê tất cả thời gian ghi bàn.', 'CLU': 'liệt kê tất cả câu lạc bộ.', 'PCA': 'liệt kê tất cả cầu thủ nhận thẻ.', 'TCA': 'liệt kê tất cả thời gian nhận thẻ.'}, 'qa_turn2': {"('CLU', 'COMP', 'CLU')": 'câu lạc bộ XXX đấu với câu lạc bộ nào?.', "('CLU', 'SCOC', 'SCO')": 'câu lạc bộ XXX đạt được bao nhiêu bàn thắng?.', "('PSC', 'SCOP', 'CLU')": 'cầu thủ XXX lập công cho câu lạc bộ nào?.', "('PSC', 'SCOT', 'TSC')": 'cầu thủ XXX lập công tại thời gian nào?', "('PCA', 'CARP', 'CLU')": 'cầu thủ XXX nhận thẻ thuộc câu lạc bộ nào?', "('PCA', 'CART', 'TCA')": 'cầu thủ XXX nhận thẻ tại thời gian nào?', "('PSI', 'SUBP', 'PSO')": 'cầu thủ XXX thay thế cho cầu thủ nào?', "('PSI', 'SUBT', 'TSI')": 'cầu thủ XXX vào sân tại thời gian nào?'}}




 14%|█▎        | 29/211 [00:00<00:01, 127.19it/s]

 22%|██▏       | 46/211 [00:00<00:01, 136.42it/s]

 31%|███       | 65/211 [00:00<00:00, 148.90it/s]

 38%|███▊      | 81/211 [00:00<00:00, 151.53it/s]

 47%|████▋     | 99/211 [00:00<00:00, 157.25it/s]

 56%|█████▌    | 118/211 [00:00<00:00, 165.10it/s]

 66%|██████▌   | 139/211 [00:00<00:00, 176.02it/s]

 75%|███████▌  | 159/211 [00:00<00:00, 180.64it/s]

 84%|████████▍ | 177/211 [00:01<00:00, 175.94it/s]

100%|██████████| 211/211 [00:01<00:00, 173.41it/s]


In [ ]:
output_dir = 'dataset'
tokenizer = AutoTokenizer.from_pretrained(argss.pretrained_model_path)
print(zalo_question_templates)
process('dev', output_dir, tokenizer, True,
        220, 45, argss.dataset_tag, 1, 45)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/141 [00:00<?, ?it/s]

  6%|▌         | 8/141 [00:00<00:01, 73.56it/s]

{'qa_turn1': {'SCO': 'liệt kê tất cả số bàn thắng.', 'PSI': 'liệt kê tất cả cầu thủ thay thế.', 'PSO': 'liệt kê tất cả cầu thủ ra sân.', 'TSI': 'liệt kê tất cả thời gian vào sân.', 'PSC': 'liệt kê tất cả cầu thủ ghi bàn.', 'TSC': 'liệt kê tất cả thời gian ghi bàn.', 'CLU': 'liệt kê tất cả câu lạc bộ.', 'PCA': 'liệt kê tất cả cầu thủ nhận thẻ.', 'TCA': 'liệt kê tất cả thời gian nhận thẻ.'}, 'qa_turn2': {"('CLU', 'COMP', 'CLU')": 'câu lạc bộ XXX đấu với câu lạc bộ nào?.', "('CLU', 'SCOC', 'SCO')": 'câu lạc bộ XXX đạt được bao nhiêu bàn thắng?.', "('PSC', 'SCOP', 'CLU')": 'cầu thủ XXX lập công cho câu lạc bộ nào?.', "('PSC', 'SCOT', 'TSC')": 'cầu thủ XXX lập công tại thời gian nào?', "('PCA', 'CARP', 'CLU')": 'cầu thủ XXX nhận thẻ thuộc câu lạc bộ nào?', "('PCA', 'CART', 'TCA')": 'cầu thủ XXX nhận thẻ tại thời gian nào?', "('PSI', 'SUBP', 'PSO')": 'cầu thủ XXX thay thế cho cầu thủ nào?', "('PSI', 'SUBT', 'TSI')": 'cầu thủ XXX vào sân tại thời gian nào?'}}




 11%|█         | 15/141 [00:00<00:01, 68.10it/s]

 16%|█▌        | 22/141 [00:00<00:01, 66.40it/s]

 20%|█▉        | 28/141 [00:00<00:01, 63.11it/s]

 26%|██▌       | 37/141 [00:00<00:01, 66.87it/s]

 33%|███▎      | 47/141 [00:00<00:01, 72.46it/s]

 39%|███▉      | 55/141 [00:00<00:01, 73.29it/s]

 45%|████▌     | 64/141 [00:00<00:01, 75.61it/s]

 52%|█████▏    | 73/141 [00:01<00:00, 77.24it/s]

 58%|█████▊    | 82/141 [00:01<00:00, 75.90it/s]

 65%|██████▌   | 92/141 [00:01<00:00, 81.56it/s]

 72%|███████▏  | 101/141 [00:01<00:00, 80.37it/s]

 79%|███████▊  | 111/141 [00:01<00:00, 84.70it/s]

 85%|████████▌ | 120/141 [00:01<00:00, 82.95it/s]

 91%|█████████▏| 129/141 [00:01<00:00, 83.79it/s]

100%|██████████| 141/141 [00:01<00:00, 78.78it/s]


In [ ]:
output_dir = 'dataset'
tokenizer = AutoTokenizer.from_pretrained(argss.pretrained_model_path)
print(zalo_question_templates)
process('train', output_dir, tokenizer, False,
        220, 45, argss.dataset_tag, 1, 90)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/560 [00:00<?, ?it/s]

  1%|          | 6/560 [00:00<00:09, 59.93it/s]

{'qa_turn1': {'SCO': 'liệt kê tất cả số bàn thắng.', 'PSI': 'liệt kê tất cả cầu thủ thay thế.', 'PSO': 'liệt kê tất cả cầu thủ ra sân.', 'TSI': 'liệt kê tất cả thời gian vào sân.', 'PSC': 'liệt kê tất cả cầu thủ ghi bàn.', 'TSC': 'liệt kê tất cả thời gian ghi bàn.', 'CLU': 'liệt kê tất cả câu lạc bộ.', 'PCA': 'liệt kê tất cả cầu thủ nhận thẻ.', 'TCA': 'liệt kê tất cả thời gian nhận thẻ.'}, 'qa_turn2': {"('CLU', 'COMP', 'CLU')": 'câu lạc bộ XXX đấu với câu lạc bộ nào?.', "('CLU', 'SCOC', 'SCO')": 'câu lạc bộ XXX đạt được bao nhiêu bàn thắng?.', "('PSC', 'SCOP', 'CLU')": 'cầu thủ XXX lập công cho câu lạc bộ nào?.', "('PSC', 'SCOT', 'TSC')": 'cầu thủ XXX lập công tại thời gian nào?', "('PCA', 'CARP', 'CLU')": 'cầu thủ XXX nhận thẻ thuộc câu lạc bộ nào?', "('PCA', 'CART', 'TCA')": 'cầu thủ XXX nhận thẻ tại thời gian nào?', "('PSI', 'SUBP', 'PSO')": 'cầu thủ XXX thay thế cho cầu thủ nào?', "('PSI', 'SUBT', 'TSI')": 'cầu thủ XXX vào sân tại thời gian nào?'}}
=================================



  2%|▎         | 14/560 [00:00<00:08, 63.64it/s]

  4%|▍         | 23/560 [00:00<00:07, 69.08it/s]

  6%|▌         | 32/560 [00:00<00:07, 72.47it/s]

  7%|▋         | 39/560 [00:00<00:07, 71.07it/s]

  9%|▉         | 50/560 [00:00<00:06, 76.21it/s]

 11%|█         | 60/560 [00:00<00:06, 80.56it/s]

 13%|█▎        | 71/560 [00:00<00:05, 87.05it/s]

 15%|█▍        | 82/560 [00:00<00:05, 90.47it/s]

 17%|█▋        | 95/560 [00:01<00:04, 98.78it/s]

 19%|█▉        | 106/560 [00:01<00:04, 97.00it/s]

 21%|██        | 116/560 [00:01<00:04, 91.63it/s]

 22%|██▎       | 126/560 [00:01<00:04, 90.18it/s]

 25%|██▍       | 139/560 [00:01<00:04, 97.86it/s]

 27%|██▋       | 150/560 [00:01<00:04, 95.46it/s]

 29%|██▉       | 162/560 [00:01<00:04, 99.24it/s]

 31%|███       | 173/560 [00:01<00:03, 99.21it/s]

 33%|███▎      | 184/560 [00:01<00:03, 97.87it/s]

 35%|███▍      | 194/560 [00:02<00:03, 91.91it/s]

 36%|███▋      | 204/560 [00:02<00:03, 92.26it/s]

Entites of relations in different sentence: 2




 38%|███▊      | 214/560 [00:02<00:03, 90.61it/s]

 40%|████      | 224/560 [00:02<00:03, 92.78it/s]

...



 42%|████▏     | 234/560 [00:02<00:03, 87.38it/s]

 43%|████▎     | 243/560 [00:02<00:03, 87.99it/s]

 45%|████▌     | 252/560 [00:02<00:03, 78.63it/s]

 47%|████▋     | 261/560 [00:02<00:03, 80.21it/s]

 48%|████▊     | 270/560 [00:03<00:03, 77.01it/s]

 50%|████▉     | 278/560 [00:03<00:03, 76.78it/s]

 51%|█████     | 286/560 [00:03<00:03, 73.59it/s]

Entites of relations in different sentence: 6




 53%|█████▎    | 295/560 [00:03<00:03, 74.18it/s]

 54%|█████▍    | 305/560 [00:03<00:03, 76.89it/s]

 56%|█████▋    | 315/560 [00:03<00:02, 81.98it/s]

 58%|█████▊    | 325/560 [00:03<00:02, 86.36it/s]

 60%|█████▉    | 334/560 [00:03<00:02, 85.02it/s]

 61%|██████▏   | 344/560 [00:03<00:02, 87.49it/s]

 63%|██████▎   | 354/560 [00:04<00:02, 89.24it/s]

 65%|██████▌   | 364/560 [00:04<00:02, 84.57it/s]

 67%|██████▋   | 373/560 [00:04<00:02, 82.88it/s]

 68%|██████▊   | 382/560 [00:04<00:02, 83.23it/s]

 70%|██████▉   | 391/560 [00:04<00:02, 81.19it/s]

 71%|███████▏  | 400/560 [00:04<00:01, 82.62it/s]

 73%|███████▎  | 409/560 [00:04<00:01, 80.81it/s]

 75%|███████▍  | 418/560 [00:04<00:01, 82.94it/s]

 76%|███████▋  | 427/560 [00:04<00:01, 83.75it/s]

 78%|███████▊  | 438/560 [00:05<00:01, 88.54it/s]

 80%|███████▉  | 447/560 [00:05<00:01, 88.36it/s]

 82%|████████▏ | 458/560 [00:05<00:01, 92.66it/s]

 84%|████████▎ | 468/560 [00:05<00:01, 65.86it/s]

 85%|████████▌ | 478/560 [00:

Entites of relations in different sentence: 2




 90%|█████████ | 506/560 [00:05<00:00, 80.29it/s]

 92%|█████████▏| 515/560 [00:06<00:00, 81.87it/s]

 94%|█████████▍| 526/560 [00:06<00:00, 87.92it/s]

 96%|█████████▌| 537/560 [00:06<00:00, 92.59it/s]

 98%|█████████▊| 547/560 [00:06<00:00, 90.13it/s]

100%|██████████| 560/560 [00:06<00:00, 85.79it/s]
